# Review Rec Bot

# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from loguru import logger
import json

import sys
sys.path.insert(0, '..')

In [3]:
import os

from tqdm.notebook import tqdm
import pandas as pd
from llama_index.core.response.notebook_utils import display_source_node

In [4]:
import nest_asyncio

nest_asyncio.apply()

In [5]:
from dotenv import load_dotenv

load_dotenv()

True

# Arguments

In [6]:
from src.run.args import RunInputArgs

ARGS = RunInputArgs(
    EXPERIMENT_NAME="Review Rec Bot - Yelp Review Rec Bot",
    RUN_NAME="028_refactor",
    RUN_DESCRIPTION="""
# Objective
Refactor code to be able to use different Vector Database and have feature flag for auto-retrieval

# Implementation

## Changelog
""",
    TESTING=False,
    LOG_TO_MLFLOW=True,
    OBSERVABILITY=True,
    RECREATE_INDEX=True,
    RECREATE_RETRIEVAL_EVAL_DATASET=False,
    RECREATE_RESPONSE_EVAL_DATASET=False,
    DEBUG=False,
)

ARGS

2024-08-17 09:46:58.389 | WARNING  | src.run.args:check_recreate_flags:33 - If RECREATE_INDEX is True, then RECREATE_RETRIEVAL_EVAL_DATASET should also be True. Automatically setting RECREATE_RETRIEVAL_EVAL_DATASET to True.
2024-08-17 09:46:58.389 | WARNING  | src.run.args:check_recreate_flags:38 - If RECREATE_INDEX is True, then RECREATE_RESPONSE_EVAL_DATASET should also be True. Automatically setting RECREATE_RESPONSE_EVAL_DATASET to True.


{
  "EXPERIMENT_NAME": "Review Rec Bot - Yelp Review Rec Bot",
  "RUN_NAME": "028_refactor",
  "RUN_DESCRIPTION": "\n# Objective\nRefactor code to be able to use different Vector Database and have feature flag for auto-retrieval\n\n# Implementation\n\n## Changelog\n",
  "TESTING": true,
  "DEBUG": false,
  "OBSERVABILITY": false,
  "LOG_TO_MLFLOW": false,
  "RECREATE_RETRIEVAL_EVAL_DATASET": true,
  "RECREATE_RESPONSE_EVAL_DATASET": true,
  "RECREATE_INDEX": true
}

# Load config

In [7]:
from src.run.cfg import RunConfig

In [8]:
cfg = RunConfig()
cfg.init(ARGS)

2024-08-17 09:46:58.436 | INFO     | src.run.cfg:init:178 - Notebook-generated artifacts are persisted at data/028_refactor
2024-08-17 09:46:58.437 | INFO     | src.run.cfg:init:184 - ARGS.RECREATE_INDEX=True -> Overwriting db_collection and storage_context_persist_dp...


In [9]:
cfg

/home/dvquys/frostmourne/study/vietai-genai03/final-project/review-rec-bot/.venv/lib/python3.11/site-packages/pydantic/main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


{
  "args": {
    "EXPERIMENT_NAME": "Review Rec Bot - Yelp Review Rec Bot",
    "RUN_NAME": "028_refactor",
    "RUN_DESCRIPTION": "\n# Objective\nRefactor code to be able to use different Vector Database and have feature flag for auto-retrieval\n\n# Implementation\n\n## Changelog\n",
    "TESTING": true,
    "DEBUG": false,
    "OBSERVABILITY": false,
    "LOG_TO_MLFLOW": false,
    "RECREATE_RETRIEVAL_EVAL_DATASET": true,
    "RECREATE_RESPONSE_EVAL_DATASET": true,
    "RECREATE_INDEX": true
  },
  "app_name": "review_rec_bot",
  "storage_context_persist_dp": "data/028_refactor/storage_context",
  "vector_db": "qdrant",
  "db_collection": "review_rec_bot__028_refactor",
  "db_collection_fp": "data/028_refactor/qdrant",
  "notebook_cache_dp": "data/028_refactor",
  "data_fp": "../data/yelp_dataset/sample/sample_100_biz/denom_review.parquet",
  "llm_cfg": {
    "llm_provider": "openai",
    "llm_model_name": "gpt-4o-mini",
    "embedding_provider": "huggingface",
    "embedding_model_

## Set up logger to collect additional info

In [10]:
collect_fp = f"{cfg.notebook_cache_dp}/collect.log"
logger.add(collect_fp, filter=lambda record: "[COLLECT]" in record['message'], mode='w')

1

# Load input data

In [11]:
data = pd.read_parquet(cfg.data_fp)
data = data.assign(
    # ChromaDB allows only str, int, float dtypes to be indexed as metadata
    # biz_categories=lambda df: df['biz_categories'].str.split(', '),
    biz_categories=lambda df: df['biz_categories'].astype(str),
    biz_attributes=lambda df: df['biz_attributes'].astype(str),
    date=lambda df: df['date'].dt.strftime('%Y-%m-%dT%H:%M:%S'),
)
logger.info(f"[COLLECT] {len(data)=}")
data.iloc[0]

2024-08-17 09:46:58.817 | INFO     | __main__:<module>:9 - [COLLECT] len(data)=5240


review_id                                      L0jv8c2FbpWSlfNC6bbUEA
user_id                                        bFPdtzu11Oi0f92EAcjqmg
business_id                                    IDtLPgUrqorrpqSLdfMhZQ
review_stars                                                        5
useful                                                              0
funny                                                               0
cool                                                                0
text                What a great addition to the Funk Zone!  Grab ...
date                                              2016-10-13T22:50:47
biz_name                                         Helena Avenue Bakery
biz_address                                     131 Anacapa St, Ste C
biz_city                                                Santa Barbara
biz_state                                                          CA
biz_postal_code                                                 93101
biz_latitude        

In [12]:
data.dtypes

review_id            object
user_id              object
business_id          object
review_stars          int64
useful                int64
funny                 int64
cool                  int64
text                 object
date                 object
biz_name             object
biz_address          object
biz_city             object
biz_state            object
biz_postal_code      object
biz_latitude        float64
biz_longitude       float64
biz_stars           float64
biz_review_count      int64
biz_is_open           int64
biz_attributes       object
biz_categories       object
biz_hours            object
dtype: object

In [13]:
input_data = data
if ARGS.TESTING:
    input_data = data[:30]  # 30 examples to avoid bm25 retreiver error...
logger.info(f"[COLLECT] {len(input_data)=}")

2024-08-17 09:46:59.155 | INFO     | __main__:<module>:4 - [COLLECT] len(input_data)=30


In [14]:
input_data.columns

Index(['review_id', 'user_id', 'business_id', 'review_stars', 'useful',
       'funny', 'cool', 'text', 'date', 'biz_name', 'biz_address', 'biz_city',
       'biz_state', 'biz_postal_code', 'biz_latitude', 'biz_longitude',
       'biz_stars', 'biz_review_count', 'biz_is_open', 'biz_attributes',
       'biz_categories', 'biz_hours'],
      dtype='object')

# Prepare documents

In [15]:
from llama_index.core import Document

documents = []
metadata_excluded_cols = ('biz_hours', 'text')
embedding_visible_metadata = ["review_stars", "biz_name", "biz_address", "biz_city", "biz_state", "biz_categories", "business_id"]
excluded_embed_metadata_keys = [k for k in input_data.columns if k not in embedding_visible_metadata]

for i, row in tqdm(input_data.iterrows(), total=len(input_data)):
    record = row.to_dict()
    text = record['text']
    metadata = {k: v for k, v in record.items() if k not in metadata_excluded_cols}
    
    doc = Document(
        text=text,
        metadata=metadata,
        excluded_embed_metadata_keys=excluded_embed_metadata_keys,
        excluded_llm_metadata_keys=excluded_embed_metadata_keys
    )
    documents.append(doc)

logger.info(f"[COLLECT] {len(documents)=}")

  0%|          | 0/30 [00:00<?, ?it/s]

2024-08-17 09:46:59.787 | INFO     | __main__:<module>:21 - [COLLECT] len(documents)=30


#### Check document embedding text

In [16]:
from llama_index.core.schema import MetadataMode

In [17]:
document = documents[0]
print(document.get_content(metadata_mode=MetadataMode.EMBED))

business_id: IDtLPgUrqorrpqSLdfMhZQ
review_stars: 5
biz_name: Helena Avenue Bakery
biz_address: 131 Anacapa St, Ste C
biz_city: Santa Barbara
biz_state: CA
biz_categories: Food, Restaurants, Salad, Coffee & Tea, Breakfast & Brunch, Sandwiches, Bakeries

What a great addition to the Funk Zone!  Grab a bite, grab some tastings, life is good. Right next door to the Santa Barbara Wine Collective, in fact it actually shares the same tables.  We had a fabulous savory croissant.


# Set up LLM

In [18]:
llm, embed_model = cfg.setup_llm()

You try to use a model that was created with version 2.7.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [19]:
print(cfg.llm_cfg.model_dump_json(indent=2))

{
  "llm_provider": "openai",
  "llm_model_name": "gpt-4o-mini",
  "embedding_provider": "huggingface",
  "embedding_model_name": "./data/finetune_embedding/finetuned_model",
  "embedding_model_dim": 768,
  "ollama__host": "192.168.100.14",
  "ollama__port": 11434
}


In [20]:
from llama_index.core import Settings
Settings.embed_model = embed_model
Settings.llm = llm

# Vector Store

In [21]:
if cfg.vector_db == 'chromadb':
    from src.run.vector_db import ChromaVectorDB as VectorDB
elif cfg.vector_db == 'qdrant':
    from src.run.vector_db import QdrantVectorDB as VectorDB

In [22]:
vector_db = VectorDB(cfg)
vector_store = vector_db.vector_store
db_collection_count = vector_db.doc_count
logger.info(f"{db_collection_count=}")

2024-08-17 09:47:04.540 | INFO     | src.run.vector_db.qdrant:_setup_db:40 - Deleting existing Qdrant collection review_rec_bot__028_refactor...
2024-08-17 09:47:04.548 | INFO     | src.run.vector_db.qdrant:_setup_db:49 - Creating new Qdrant collection review_rec_bot__028_refactor...
Both client and aclient are provided. If using `:memory:` mode, the data between clients is not synced.
2024-08-17 09:47:04.968 | INFO     | __main__:<module>:4 - db_collection_count=0


# Index Embeddings

In [23]:
import time
import pickle
from multiprocessing import set_start_method
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext
# from llama_index.core.node_parser import SentenceSplitter

In [24]:
chunker = SemanticSplitterNodeParser
chunker_cfg = {
    "buffer_size": 1,
    "breakpoint_percentile_threshold": 95,
    "embed_model": embed_model
}

# chunker = SentenceSplitter
# chunker_cfg = {
#     "chunk_size": 512,
#     "chunk_overlap": 10
# }

In [25]:
t0 = time.perf_counter()

if db_collection_count > 0 and ARGS.RECREATE_INDEX == False:
    logger.info(f"Loading Storage Context from {cfg.storage_context_persist_dp}...")
    docstore = SimpleDocumentStore.from_persist_dir(persist_dir=cfg.storage_context_persist_dp)
    storage_context = StorageContext.from_defaults(
        docstore=docstore,
        vector_store=vector_store
    )
    nodes = list(docstore.docs.values())
else:
    logger.info(f"Creating new DB index...")
    from llama_index.core.extractors import TitleExtractor
    from llama_index.core.ingestion import IngestionPipeline, IngestionCache

    # create the pipeline with transformations
    pipeline = IngestionPipeline(
        transformations=[
            chunker(**chunker_cfg),
            embed_model,
        ],
        vector_store = vector_store
    )

    num_workers = None
    # TODO: I can get the preprocessing completed much quicker if I push the initial steps through a multiprocessing loop and then separately create the embeddings using the built-in batching already provided in the SentenceTransformer encode method.
    # Ref: https://github.com/run-llama/llama_index/issues/10104#issuecomment-1899401584
    # Currently setting num_workers leads to code simple hang
    # Ref: https://github.com/run-llama/llama_index/issues/10104
    # num_workers = os.cpu_count() - 1
    # os.environ['TOKENIZERS_PARALLELISM'] = 'true'
    # set_start_method("spawn", force=True)  # it hangs without this line
    logger.info(f"Running Ingestion Pipeline with {num_workers=}...")
    nodes = await pipeline.arun(documents=documents, num_workers=num_workers, show_progress=True)

    # Define Docstore as an abstraction on top of nodes to easily manage (e.g. get node by id)
    docstore = SimpleDocumentStore()
    await docstore.async_add_documents(nodes)
    storage_context = StorageContext.from_defaults(
        docstore=docstore,
        vector_store=vector_store
    )
    logger.info(f"Persisting Storage Context to {cfg.storage_context_persist_dp}...")
    storage_context.persist(cfg.storage_context_persist_dp)

t1 = time.perf_counter()

2024-08-17 09:47:05.062 | INFO     | __main__:<module>:12 - Creating new DB index...
2024-08-17 09:47:05.068 | INFO     | __main__:<module>:33 - Running Ingestion Pipeline with num_workers=None...


Parsing nodes:   0%|          | 0/30 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/4 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/4 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/9 [00:00<?, ?it/s]

...

Generating embeddings:   0%|          | 0/6 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/4 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/17 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

Generating embeddings: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 24.37it/s]
2024-08-17 09:47:06.935 | INFO     | __main__:<module>:43 - Persisting Storage Context to data/028_refactor/storage_context...


In [26]:
logger.info(f"Indexing {len(documents)} documents into VectorStoreIndex took {t1 - t0:,.0f}s")
logger.info(f"[COLLECT] {len(nodes)=}")

2024-08-17 09:47:06.988 | INFO     | __main__:<module>:1 - Indexing 30 documents into VectorStoreIndex took 2s
2024-08-17 09:47:06.989 | INFO     | __main__:<module>:2 - [COLLECT] len(nodes)=60


# Analyze Chunks

In [27]:
for i, node in enumerate(nodes[:5]):
    print(f"\n\n==========Node {i+1}============")
    print(node.metadata)
    print(node.get_text())



==========Node 1============
{'review_id': 'L0jv8c2FbpWSlfNC6bbUEA', 'user_id': 'bFPdtzu11Oi0f92EAcjqmg', 'business_id': 'IDtLPgUrqorrpqSLdfMhZQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2016-10-13T22:50:47', 'biz_name': 'Helena Avenue Bakery', 'biz_address': '131 Anacapa St, Ste C', 'biz_city': 'Santa Barbara', 'biz_state': 'CA', 'biz_postal_code': '93101', 'biz_latitude': 34.4144445, 'biz_longitude': -119.6906718, 'biz_stars': 4.0, 'biz_review_count': 389, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'touristy\': False, \'hipster\': True, \'romantic\': False, \'divey\': False, \'intimate\': False, \'trendy\': True, \'upscale\': False, \'classy\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'l

# Construct Retriever

In [32]:
# from src.run.retriever.auto_retrieval import get_retriever
from src.run.retriever.retriever import get_retriever

In [33]:
index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)
vector_retriever = get_retriever(cfg, index)

In [34]:
query = "Nashville, grocery"
vector_retrieval_results = vector_retriever.retrieve(query)
for result in vector_retrieval_results:
    display_source_node(result, source_length=1000, show_source_metadata=True)

**Node ID:** 28874a44-693f-449a-8695-8856c4bd9b47<br>**Similarity:** 0.5073119<br>**Text:** Skip the fancy desert after dinner to go here, it's an excuse to walk around Broadway.  The homemade ice cream is really good.<br>**Metadata:** {'review_id': 'VVvmX64sw8QS837kuQIXCw', 'user_id': 'K6GM4-fMcVPG77lzgpOHeQ', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 4, 'useful': 1, 'funny': 0, 'cool': 0, 'date': '2009-03-01T10:27:17', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': \'False\', \'Caters\': \'False\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': \'False\', \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'None\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Ice Cream & Frozen Yogurt, Coffee & Tea, Restaurants, Sandwiches, Food'}<br>

**Node ID:** a187a8f4-7fa6-45fe-8d72-d6a9c1921315<br>**Similarity:** 0.48538125<br>**Text:** Not much else to say but an amazing ice cream store that others should try to emulate.<br>**Metadata:** {'review_id': '6odPSjoPWOyeDQpgJbdGaA', 'user_id': 'jLr7vcDg43TqS2OK4ROvPQ', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2016-03-30T23:22:54', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': \'False\', \'Caters\': \'False\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': \'False\', \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'None\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Ice Cream & Frozen Yogurt, Coffee & Tea, Restaurants, Sandwiches, Food'}<br>

**Node ID:** 7cb6e5ef-5312-4bda-9360-1c13382a3f5c<br>**Similarity:** 0.47831538<br>**Text:** Yum!<br>**Metadata:** {'review_id': 'BeUP5QUgnF3CHelCJtw1Lw', 'user_id': 'tbllMvbVFAPO2E7n19nBXA', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 4, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2014-09-09T18:19:48', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': \'False\', \'Caters\': \'False\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': \'False\', \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'None\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Ice Cream & Frozen Yogurt, Coffee & Tea, Restaurants, Sandwiches, Food'}<br>

**Node ID:** 6a5c4e6f-b9ba-4cd5-8e66-00bb602fc702<br>**Similarity:** 0.47577637<br>**Text:** We stopped here's on the recommendation of our shuttle driver.  The red velvet ice cream is wonderful!! Pieces of cake. Creamy goodness.<br>**Metadata:** {'review_id': 'BeUP5QUgnF3CHelCJtw1Lw', 'user_id': 'tbllMvbVFAPO2E7n19nBXA', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 4, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2014-09-09T18:19:48', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': \'False\', \'Caters\': \'False\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': \'False\', \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'None\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Ice Cream & Frozen Yogurt, Coffee & Tea, Restaurants, Sandwiches, Food'}<br>

**Node ID:** 0ad25b89-ffe3-4f5f-8e04-bd6a1a6926ba<br>**Similarity:** 0.46982992<br>**Text:** Quality ice cream here. There was a line this Thursday late but it moved quick. Lots of tasty sounding choices, but I chose the Loaded Butterscotch and the Nutter Butter on a chocolate waffle cone. No complaints!<br>**Metadata:** {'review_id': '8ZhvFEoqBPrmN1DDVJz_uw', 'user_id': 'MqcfaSdLXh7H1Aey_HPUsw', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 4, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2018-03-16T03:05:05', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': \'False\', \'Caters\': \'False\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': \'False\', \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'None\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Ice Cream & Frozen Yogurt, Coffee & Tea, Restaurants, Sandwiches, Food'}<br>

...

**Node ID:** 721f5a26-6d99-4f75-bfac-82aa4a3cccd2<br>**Similarity:** 0.041281275<br>**Text:** Bunch of high school/college kids running the place.<br>**Metadata:** {'review_id': 'NmrPTpc8hMk7Py1WXCGAyQ', 'user_id': 'XOa3fuOw90GWvghLHyTQYA', 'business_id': 'kV_Q1oqis8Qli8dUoGpTyQ', 'review_stars': 2, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2018-08-05T00:10:12', 'biz_name': 'Ardmore Pizza', 'biz_address': '10 Rittenhouse Pl', 'biz_city': 'Ardmore', 'biz_state': 'PA', 'biz_postal_code': '19003', 'biz_latitude': 40.0067071, 'biz_longitude': -75.289671, 'biz_stars': 3.5, 'biz_review_count': 109, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'classy\': False, \'hipster\': False, \'divey\': False, \'touristy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'True\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': None, \'DriveThru\': None, \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': "{\'dessert\': False, \'latenight\': True, \'lunch\': False, \'dinner\': True, \'brunch\': False, \'breakfast\': False}", \'HappyHour\': None, \'HasTV\': \'True\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'False\', \'RestaurantsAttire\': "u\'casual\'", \'RestaurantsDelivery\': \'True\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'1\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "u\'no\'"}', 'biz_categories': 'Pizza, Restaurants'}<br>

**Node ID:** 51081086-6134-44da-a070-8b91cfb798b4<br>**Similarity:** 0.040380143<br>**Text:** They are very soft and the outside is chewy, and they taste great with just about anything: butter, hummus, melted cheese, or peanut butter & jam. The only problem is that I can't eat them, because they give me really bad eczema on my face. I suspect it's all the yeast they put in it. The focaccia gluten-free bread they sell is also really good. This is like a thick flat bread that is flavoured with tons of herbs and I think some garlic too. It's so good, and although it contains yeast, I don't get eczema, so it must not be that much. 

These breads are very expensive though, so I only get them as a treat.<br>**Metadata:** {'review_id': 'vpPUmQz-UAJjaulqMNZW5w', 'user_id': 'l9be6lRlJhumEM8ruMMJ7g', 'business_id': '5BmQX4UVJY19mMtafMg7JA', 'review_stars': 4, 'useful': 6, 'funny': 0, 'cool': 0, 'date': '2008-12-04T08:03:31', 'biz_name': 'Breadland Organic Whole Grain Bakery', 'biz_address': '11642 104 Avenue, Unit 642', 'biz_city': 'Edmonton', 'biz_state': 'AB', 'biz_postal_code': 'T5K 2T7', 'biz_latitude': 53.547205, 'biz_longitude': -113.522448, 'biz_stars': 4.0, 'biz_review_count': 23, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': None, \'Ambience\': None, \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': None, \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': None, \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': None, \'DriveThru\': None, \'GoodForDancing\': None, \'GoodForKids\': None, \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': None, \'Music\': None, \'NoiseLevel\': None, \'OutdoorSeating\': None, \'RestaurantsAttire\': None, \'RestaurantsDelivery\': \'False\', \'RestaurantsGoodForGroups\': None, \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': None, \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': None}', 'biz_categories': 'Specialty Food, Bakeries, Food, Health Markets'}<br>

**Node ID:** 1409aebb-9c41-4695-94b6-f21a211cdd9a<br>**Similarity:** 0.03138837<br>**Text:** Can I say it- FINALLY! Am amazing place for sushi in the city. We were there for what they were calling their soft opening, where apparently they are testing out menu and cocktail items for their "official" opening sometime soon. We were very pleased with the ambiance, service and food/drink. The fish is top quality. We said hello to the owner/executive chef and he seemed to be beaming over his accomplishments, rightfully so! This is a must stop if you're a sushi/Asian fusion fan. We look forward to seeing what's in store for Tuna Bar and are thrilled to have this a two minute walk away!<br>**Metadata:** {'review_id': 'XYaDbPKyJAu4k2aUOIth5g', 'user_id': 'Qsk0aTclam9W_DIK6bx42A', 'business_id': 'MUTTqe8uqyMdBl186RmNeA', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2017-12-16T00:13:06', 'biz_name': 'Tuna Bar', 'biz_address': '205 Race St', 'biz_city': 'Philadelphia', 'biz_state': 'PA', 'biz_postal_code': '19106', 'biz_latitude': 39.953949, 'biz_longitude': -75.1432262, 'biz_stars': 4.0, 'biz_review_count': 245, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "\'full_bar\'", \'Ambience\': "{\'touristy\': False, \'hipster\': False, \'romantic\': False, \'divey\': False, \'intimate\': False, \'trendy\': False, \'upscale\': False, \'classy\': True, \'casual\': False}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': None, \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{u\'valet\': False, u\'garage\': None, u\'street\': True, u\'lot\': False, u\'validated\': None}", \'ByAppointmentOnly\': None, \'Caters\': None, \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': None, \'GoodForDancing\': None, \'GoodForKids\': \'False\', \'GoodForMeal\': "{\'dessert\': True, \'latenight\': None, \'lunch\': None, \'dinner\': True, \'brunch\': None, \'breakfast\': None}", \'HappyHour\': \'True\', \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'True\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'True\', \'RestaurantsTableService\': \'True\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': \'True\', \'WiFi\': "\'free\'"}', 'biz_categories': 'Sushi Bars, Restaurants, Japanese'}<br>

**Node ID:** 22d25f57-1dfb-4262-ae9e-e4f8f66e0dc0<br>**Similarity:** 0.028304562<br>**Text:** The food.  THE FOOD!!!!  We had their summer special .  Holy COW, were they fantastic!  Entrees were   seafood shrimp Alfredo and our friend and I had the steak.  Honestly, you cannot go wrong with anything that you order from Tony's.  Top notch, old school dining experience, which is RARE.<br>**Metadata:** {'review_id': 'uiqzlDEsUN_y1awEw_HHDA', 'user_id': 'qmQPWMV_YYmwV2DyvmIDYQ', 'business_id': 'ljxNT9p0y7YMPx0fcNBGig', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2018-07-17T01:07:49', 'biz_name': "Tony's Restaurant & 3rd Street Cafe", 'biz_address': '312 Piasa St', 'biz_city': 'Alton', 'biz_state': 'IL', 'biz_postal_code': '62002', 'biz_latitude': 38.896563, 'biz_longitude': -90.1862032987, 'biz_stars': 3.0, 'biz_review_count': 94, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'full_bar\'", \'Ambience\': "{\'touristy\': False, \'hipster\': False, \'romantic\': False, \'divey\': False, \'intimate\': False, \'trendy\': False, \'upscale\': False, \'classy\': True, \'casual\': True}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': \'False\', \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'True\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': None, \'DriveThru\': None, \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': "{\'dessert\': False, \'latenight\': False, \'lunch\': False, \'dinner\': True, \'brunch\': False, \'breakfast\': False}", \'HappyHour\': \'True\', \'HasTV\': \'True\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "u\'casual\'", \'RestaurantsDelivery\': \'False\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'True\', \'RestaurantsTableService\': \'True\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': \'True\', \'WiFi\': "\'free\'"}', 'biz_categories': 'Restaurants, Specialty Food, Steakhouses, Food, Italian, Pizza, Pasta Shops'}<br>

**Node ID:** f760c19e-041e-4d5f-8a05-2802d9f39f99<br>**Similarity:** 0.028154101<br>**Text:** Food used to great years ago but getting really chintzy.  Beef with broccoli, thin tough pieces of beef, six slices covered in veggies.<br>**Metadata:** {'review_id': 'Q8k6bndVPPDAQl9KuU9POA', 'user_id': '1HUAEUkVOLZ5DptFf7oIvQ', 'business_id': 'wghnIlMb_i5U46HMBGx9ig', 'review_stars': 2, 'useful': 1, 'funny': 0, 'cool': 0, 'date': '2016-02-23T18:05:34', 'biz_name': 'China Dragon Restaurant', 'biz_address': '1625 W Valencia Rd, Ste 101-103', 'biz_city': 'Tucson', 'biz_state': 'AZ', 'biz_postal_code': '85746', 'biz_latitude': 32.1323047, 'biz_longitude': -110.9999851, 'biz_stars': 3.0, 'biz_review_count': 23, 'biz_is_open': 0, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "\'beer_and_wine\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'classy\': False, \'hipster\': False, \'divey\': False, \'touristy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': None, \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'False\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': None, \'DriveThru\': None, \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'quiet\'", \'OutdoorSeating\': \'False\', \'RestaurantsAttire\': "u\'casual\'", \'RestaurantsDelivery\': \'False\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'True\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "\'no\'"}', 'biz_categories': 'Restaurants, Chinese'}<br>

In [33]:
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer

bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=cfg.retrieval_cfg.retrieval_sparse_top_k,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

bm25_retrieval_results = bm25_retriever.retrieve("Nashville, grocery")
for result in bm25_retrieval_results:
    display_source_node(result, source_length=1000, show_source_metadata=True)

DEBUG [bm25s] Building index from IDs objects


**Node ID:** 80425930-ad09-402b-915a-dbb7abdd60c9<br>**Similarity:** 3.2125637531280518<br>**Text:** Convenient grocery store. A bit ghetto, and not because of it's location at all.<br>**Metadata:** {'review_id': '-kdFaer2K56NJCGNadTe5w', 'user_id': 'UYgAC9mvMYc12Ufc5KVUnA', 'business_id': '-4dYswJy7SPcbcERvitmIg', 'review_stars': 3, 'useful': 1, 'funny': 1, 'cool': 0, 'date': '2010-10-27T15:07:05', 'biz_name': 'Pathmark', 'biz_address': '3021 Grays Ferry Ave', 'biz_city': 'Philadelphia', 'biz_state': 'PA', 'biz_postal_code': '19146', 'biz_latitude': 39.9404026, 'biz_longitude': -75.1932966, 'biz_stars': 2.5, 'biz_review_count': 34, 'biz_is_open': 0, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': None, \'Ambience\': None, \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': None, \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': None, \'DriveThru\': None, \'GoodForDancing\': None, \'GoodForKids\': None, \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': None, \'Music\': None, \'NoiseLevel\': None, \'OutdoorSeating\': None, \'RestaurantsAttire\': None, \'RestaurantsDelivery\': None, \'RestaurantsGoodForGroups\': None, \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': None, \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': None, \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': None}', 'biz_categories': 'Food, Grocery'}<br>

**Node ID:** 6bd328a8-5d13-492f-bd84-0597c34fd2c5<br>**Similarity:** 3.1722183227539062<br>**Text:** A good grocery store, just requires a car to get to (or bus). This used to be somewhere they shuttled local university students from Penn to come get groceries; not sure if they still do that.

There's almost always a good deal to be had, fresh produce and meat, and feels like a truly suburban grocery store. Lines are long at times, but the only real problem is the distance from the city if you don't have a car.<br>**Metadata:** {'review_id': 'hKTTE9mOS1tXI6gDbDU_Mw', 'user_id': 'nnwBdqGHIAJQ5QX9lHOtrQ', 'business_id': '-4dYswJy7SPcbcERvitmIg', 'review_stars': 3, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2009-01-20T16:11:51', 'biz_name': 'Pathmark', 'biz_address': '3021 Grays Ferry Ave', 'biz_city': 'Philadelphia', 'biz_state': 'PA', 'biz_postal_code': '19146', 'biz_latitude': 39.9404026, 'biz_longitude': -75.1932966, 'biz_stars': 2.5, 'biz_review_count': 34, 'biz_is_open': 0, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': None, \'Ambience\': None, \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': None, \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': None, \'DriveThru\': None, \'GoodForDancing\': None, \'GoodForKids\': None, \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': None, \'Music\': None, \'NoiseLevel\': None, \'OutdoorSeating\': None, \'RestaurantsAttire\': None, \'RestaurantsDelivery\': None, \'RestaurantsGoodForGroups\': None, \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': None, \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': None, \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': None}', 'biz_categories': 'Food, Grocery'}<br>

**Node ID:** e9156481-4daa-495e-a77d-eae6b7381c07<br>**Similarity:** 3.0762240886688232<br>**Text:** This grocery store SUCKS!  But it's the closest one to my house, so I go there for things that I need fast.  The produce is the worst possible.  I don't think the prices are very good, but then again it seems that all grocery stores in Philly are outrageously priced.  Employees are the slowest moving in the world!<br>**Metadata:** {'review_id': 'krw5FlAWXS16NkkqViK-wg', 'user_id': 'B-d6QdtkkoLHCyIyuT6y4A', 'business_id': '-4dYswJy7SPcbcERvitmIg', 'review_stars': 2, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2011-05-12T20:29:04', 'biz_name': 'Pathmark', 'biz_address': '3021 Grays Ferry Ave', 'biz_city': 'Philadelphia', 'biz_state': 'PA', 'biz_postal_code': '19146', 'biz_latitude': 39.9404026, 'biz_longitude': -75.1932966, 'biz_stars': 2.5, 'biz_review_count': 34, 'biz_is_open': 0, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': None, \'Ambience\': None, \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': None, \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': None, \'DriveThru\': None, \'GoodForDancing\': None, \'GoodForKids\': None, \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': None, \'Music\': None, \'NoiseLevel\': None, \'OutdoorSeating\': None, \'RestaurantsAttire\': None, \'RestaurantsDelivery\': None, \'RestaurantsGoodForGroups\': None, \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': None, \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': None, \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': None}', 'biz_categories': 'Food, Grocery'}<br>

**Node ID:** 0aba2883-bbee-4857-b997-97f86bf4ce37<br>**Similarity:** 3.0317909717559814<br>**Text:** Free parking garage.  Eat in grocery store or deli type restaurant....everything is by the pound.<br>**Metadata:** {'review_id': 'vQxUaNanrYGZ_X3qCiIC3g', 'user_id': 'btAiOvMIfAbP5Sc5fAR8zA', 'business_id': 'lk9IwjZXqUMqqOhM774DtQ', 'review_stars': 4, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2017-07-08T18:07:49', 'biz_name': 'Caviar & Bananas', 'biz_address': '2031 Broadway', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37203', 'biz_latitude': 36.1483712, 'biz_longitude': -86.7988947, 'biz_stars': 3.5, 'biz_review_count': 159, 'biz_is_open': 0, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'beer_and_wine\'", \'Ambience\': "{\'touristy\': False, \'hipster\': False, \'romantic\': False, \'divey\': False, \'intimate\': False, \'trendy\': True, \'upscale\': False, \'classy\': True, \'casual\': True}", \'BYOB\': None, \'BestNights\': "{\'monday\': False, \'tuesday\': False, \'friday\': False, \'wednesday\': False, \'thursday\': False, \'sunday\': False, \'saturday\': False}", \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': True, \'street\': False, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'True\', \'CoatCheck\': \'False\', \'Corkage\': None, \'DogsAllowed\': None, \'DriveThru\': None, \'GoodForDancing\': \'False\', \'GoodForKids\': \'True\', \'GoodForMeal\': "{\'dessert\': True, \'latenight\': False, \'lunch\': True, \'dinner\': False, \'brunch\': True, \'breakfast\': True}", \'HappyHour\': \'True\', \'HasTV\': \'False\', \'Music\': "{\'dj\': False, \'background_music\': False, \'no_music\': False, \'jukebox\': False, \'live\': False, \'video\': False, \'karaoke\': False}", \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'True\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'False\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': "u\'no\'", \'WheelchairAccessible\': \'True\', \'WiFi\': "\'free\'"}', 'biz_categories': 'Coffee & Tea, Restaurants, Wine Bars, Bars, Nightlife, American (Traditional), Event Planning & Services, Food, Caterers, Breakfast & Brunch, Cafes, Diners'}<br>

...

**Node ID:** 5d93913c-e1b1-410c-918a-5e092693f27b<br>**Similarity:** 2.1714556217193604<br>**Text:** Handcrafted ice cream right in the heart of Nashville. The place is always busy but worth the wait.<br>**Metadata:** {'review_id': '073JO4AkBwFz-6VoFSB--A', 'user_id': 'wa1PJGn1XxhIFbGWTeIbOQ', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2019-07-07T22:57:17', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': \'False\', \'Caters\': \'False\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': \'False\', \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'None\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Ice Cream & Frozen Yogurt, Coffee & Tea, Restaurants, Sandwiches, Food'}<br>

**Node ID:** 97656e33-c1c6-41d2-8fc0-e54b899131ac<br>**Similarity:** 2.1714556217193604<br>**Text:** I had the Nashville Hot Chicken Salad and it was wonderful! Big enough to eat for lunch and dinner!<br>**Metadata:** {'review_id': 'R2AI3QGNuWNP2sHPd40ewg', 'user_id': '1xY7RkCcAzS-KBhZ0ZEE0w', 'business_id': 'lk9IwjZXqUMqqOhM774DtQ', 'review_stars': 4, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2019-10-23T18:24:14', 'biz_name': 'Caviar & Bananas', 'biz_address': '2031 Broadway', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37203', 'biz_latitude': 36.1483712, 'biz_longitude': -86.7988947, 'biz_stars': 3.5, 'biz_review_count': 159, 'biz_is_open': 0, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'beer_and_wine\'", \'Ambience\': "{\'touristy\': False, \'hipster\': False, \'romantic\': False, \'divey\': False, \'intimate\': False, \'trendy\': True, \'upscale\': False, \'classy\': True, \'casual\': True}", \'BYOB\': None, \'BestNights\': "{\'monday\': False, \'tuesday\': False, \'friday\': False, \'wednesday\': False, \'thursday\': False, \'sunday\': False, \'saturday\': False}", \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': True, \'street\': False, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'True\', \'CoatCheck\': \'False\', \'Corkage\': None, \'DogsAllowed\': None, \'DriveThru\': None, \'GoodForDancing\': \'False\', \'GoodForKids\': \'True\', \'GoodForMeal\': "{\'dessert\': True, \'latenight\': False, \'lunch\': True, \'dinner\': False, \'brunch\': True, \'breakfast\': True}", \'HappyHour\': \'True\', \'HasTV\': \'False\', \'Music\': "{\'dj\': False, \'background_music\': False, \'no_music\': False, \'jukebox\': False, \'live\': False, \'video\': False, \'karaoke\': False}", \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'True\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'False\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': "u\'no\'", \'WheelchairAccessible\': \'True\', \'WiFi\': "\'free\'"}', 'biz_categories': 'Coffee & Tea, Restaurants, Wine Bars, Bars, Nightlife, American (Traditional), Event Planning & Services, Food, Caterers, Breakfast & Brunch, Cafes, Diners'}<br>

**Node ID:** e252bf71-ac1b-4d2d-b29d-829d6724b7de<br>**Similarity:** 2.162130832672119<br>**Text:** the cocktails were great & we ordered plenty of hot sake to enjoy our evening. the main reason why I am giving it 3 stars is because the menu is a little pricey (location is downtown nashville off 3rd) & nothing spectacular stood out for it being a new spot in nashville.<br>**Metadata:** {'review_id': 'OI6hq_7xWuK4JuFpkw_nqw', 'user_id': 'lU_H3xHRiWFkVStKX9sd9g', 'business_id': 'tMkwHmWFUEXrC9ZduonpTg', 'review_stars': 3, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2018-12-16T01:36:24', 'biz_name': 'The Green Pheasant', 'biz_address': '215 1st Ave S', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1598858, 'biz_longitude': -86.7731974, 'biz_stars': 4.0, 'biz_review_count': 161, 'biz_is_open': 0, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'full_bar\'", \'Ambience\': "{\'touristy\': False, \'hipster\': False, \'romantic\': False, \'divey\': False, \'intimate\': False, \'trendy\': False, \'upscale\': False, \'classy\': True, \'casual\': False}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': \'False\', \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': True, \'street\': False, \'validated\': True, \'lot\': False, \'valet\': True}", \'ByAppointmentOnly\': None, \'Caters\': \'True\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': None, \'GoodForDancing\': None, \'GoodForKids\': \'False\', \'GoodForMeal\': "{\'dessert\': None, \'latenight\': None, \'lunch\': None, \'dinner\': True, \'brunch\': None, \'breakfast\': None}", \'HappyHour\': \'True\', \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'False\', \'RestaurantsAttire\': "u\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': None, \'RestaurantsReservations\': \'True\', \'RestaurantsTableService\': \'True\', \'RestaurantsTakeOut\': \'None\', \'Smoking\': None, \'WheelchairAccessible\': \'True\', \'WiFi\': "u\'free\'"}', 'biz_categories': 'Restaurants, Japanese, Seafood'}<br>

**Node ID:** 94ef60a9-6538-46a2-bad1-c6464379ca23<br>**Similarity:** 2.1461713314056396<br>**Text:** Stopped here on a whim for lunch on our way home to Orlando after a weekend at the beach. Delicious sandwiches, lasagna, ziti and salad. Bought some wonderful Italian groceries including fresh mozzarella  and fresh bread to take home. Very friendly place...family business.<br>**Metadata:** {'review_id': 'LC8mOm9RulAdyNlI9a2GNQ', 'user_id': 'UwVqHjSqTRz3FUgr2TPsKg', 'business_id': '0bPLkL0QhhPO5kt1_EXmNQ', 'review_stars': 5, 'useful': 1, 'funny': 0, 'cool': 0, 'date': '2015-12-30T20:36:26', 'biz_name': "Zio's Italian Market", 'biz_address': '2575 E Bay Dr', 'biz_city': 'Largo', 'biz_state': 'FL', 'biz_postal_code': '33771', 'biz_latitude': 27.9161159, 'biz_longitude': -82.7604608, 'biz_stars': 4.5, 'biz_review_count': 100, 'biz_is_open': 0, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': \'False\', \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'True\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': None, \'DriveThru\': None, \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': "{\'dessert\': False, \'latenight\': False, \'lunch\': True, \'dinner\': False, \'brunch\': False, \'breakfast\': False}", \'HappyHour\': None, \'HasTV\': \'True\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'False\', \'RestaurantsAttire\': "u\'casual\'", \'RestaurantsDelivery\': \'True\', \'RestaurantsGoodForGroups\': \'False\', \'RestaurantsPriceRange2\': \'1\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'False\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': \'True\', \'WiFi\': "u\'no\'"}', 'biz_categories': 'Food, Delis, Italian, Bakeries, Restaurants'}<br>

**Node ID:** 4b4aefc7-c299-4a4e-917a-43cf53a3caea<br>**Similarity:** 2.142127275466919<br>**Text:** After visiting Mike's Ice Cream in Nashville, I am now saddened that my favorite spot is across the country. The home made Vanilla instantly reminded me of my youth when my dad used to make ice cream in the summer. If you are in Nashville, I highly recommend Mike's.<br>**Metadata:** {'review_id': 'rtUBWJgwUptv1fvulsGmnQ', 'user_id': 'E1Gfpm8X0JB-x0E7Gsa2qg', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2012-06-01T21:50:16', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': \'False\', \'Caters\': \'False\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': \'False\', \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'None\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Ice Cream & Frozen Yogurt, Coffee & Tea, Restaurants, Sandwiches, Food'}<br>

In [34]:
# TODO: Reconsider this test.
# Currently it aims to make sure the vector retriever, 
# when given a piece of text extracted from a node, 
# would return the original node.
# But what is the value of it?

# test_query = nodes[3].text
test_query = "Received confirmation of my order at 5 pm"
vector_results = vector_retriever.retrieve(test_query)
bm25_results = bm25_retriever.retrieve(test_query)

# Check if the original node in top 10:
found = False
for vector_result in vector_results[:10]:
    target_id = bm25_results[0].id_
    if vector_result.id_ == target_id:
        found = True
        break
# assert found, "Can not find the original node in the top 10 vector results"
# assert bm25_results[0].id_ == vector_results[0].id_, "Different vector_retriever nodes vs bm25_retriever nodes"

In [35]:
from llama_index.core.retrievers import QueryFusionRetriever

# query_gen_prompt = """
# You are a helpful assistant that expands an input query into new strings that aim to increase the recall of an information retrieval system. The strings can be queries or paragraphs or sentences.
# You should apply different techniques to create new strings. Here are some example techniques:
# - Technique 1 - Optimize for full-text search: Extract the key concepts and entities as keywords. This technique can generate multiple strings instead of just one. Example input query: "What are some places to enjoy cold brew coffee in Hanoi?" -> Expected output:  string 1 = "cold brew coffee", string 2 = "hanoi". Do not modify the keyword, keep the keyword exactly in the input query. Return each output string on a new line.
# - Technique 2 - Optimize for similarity-based vector retrieval: Create a fake user review that should contain the answer for the question. Example input query: "What are some good Pho restaurants in Singapore?" -> Expected output query: "I found So Pho offerring a variety of choices to enjoy not Pho but some other Vietnamese dishes like bun cha. The price is reasonable."
# - Technique 3 - Increase recall: Paraphase the original question into a new question but use different synonyms or broader concepts. Example input query: "How to find some good local grocery stores near Changi Singapore?" -> Expected output query: "Where to find a good mart selling Singaporean stuffs to the east of Singapore?"

# Generate at least {num_queries} new strings by iterating over the technique in order. For example, your first generated string should always use technique 1, second technique 2, third technique 3. If run of of techniques then re-iterate from the start.

# Return one string on each line, related to the input query.

# Only return the strings. Never include the chosen technique.

# Input Query: {query}\n
# New strings:\n
# """

# query_gen_prompt = """
# You are a helpful assistant that expands an input query into new strings that aim to increase the recall of an information retrieval system. The strings can be queries or paragraphs or sentences.
# You should apply different techniques to create new strings. Here are some example techniques:
# - Technique 1 - Optimize for full-text search: Rephrase the input query to contain only important keywords. Remove all stopwords and low information words. Example input query: "What are some places to enjoy cold brew coffee in Hanoi?" -> Expected output:  "cold brew coffee hanoi"
# - Technique 2 - Optimize for similarity-based vector retrieval: Create a fake user review that should contain the answer for the question. Example input query: "What are some good Pho restaurants in Singapore?" -> Expected output query: "I found So Pho offerring a variety of choices to enjoy not Pho but some other Vietnamese dishes like bun cha. The price is reasonable."
# - Technique 3 - Increase recall: Paraphase the original question into a new question but use different synonyms or broader concepts. Example input query: "How to find some good local grocery stores near Changi Singapore?" -> Expected output query: "Where to find a good mart selling Singaporean stuffs to the east of Singapore?"

# Generate at least {num_queries} new strings by iterating over the technique in order. For example, your first generated string should always use technique 1, second technique 2, third technique 3. If run of of techniques then re-iterate from the start.

# Return one string on each line, related to the input query.

# Only return the strings. Never include the chosen technique.

# Input Query: {query}\n
# New strings:\n
# """

query_gen_prompt = """
You are a helpful assistant that expands an input query into new strings that aim to increase the recall of an information retrieval system. The strings can be queries or paragraphs or sentences.
You should apply different techniques to create new strings. Here are some example techniques:
- Technique 1 - Optimize for full-text search: Rephrase the input query to contain only important keywords. Remove all stopwords and low information words. Example input query: "What are some places to enjoy cold brew coffee in Hanoi?" -> Expected output:  "cold brew coffee hanoi"
- Technique 2 - Optimize for similarity-based vector retrieval: Create a fake user review that should contain the answer for the question. Example input query: "What are some good Pho restaurants in Singapore?" -> Expected output query: "I found So Pho offerring a variety of choices to enjoy not Pho but some other Vietnamese dishes like bun cha. The price is reasonable."

Generate at least {num_queries} new strings by iterating over the technique in order. For example, your first generated string should always use technique 1, second technique 2. If run of of techniques then re-iterate from the start.

Return one string on each line, related to the input query.

Only return the strings. Never include the chosen technique.

Input Query: {query}\n
New strings:\n
"""

from llama_index.llms.openai import OpenAI
llm = OpenAI(model=cfg.eval_cfg.response_eval_llm_model, **cfg.eval_cfg.response_eval_llm_model_config)

retriever = QueryFusionRetriever(
    [vector_retriever, bm25_retriever],
    llm=llm,
    similarity_top_k=cfg.retrieval_cfg.retrieval_top_k,
    num_queries=2,  # set this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    query_gen_prompt=query_gen_prompt
)

## Test retrieval

In [36]:
query = "What are some highly reviewed places to enjoy cold brew coffee in Santa Barbara?"
retrieval_results = await retriever.aretrieve(query)
for node in retrieval_results:
    display_source_node(node, source_length=1000)
    print(node.get_content(metadata_mode=MetadataMode.LLM))
    print("\n")

Generated queries:
cold brew coffee Santa Barbara
"I recently visited a café in Santa Barbara that had the best cold brew coffee I've ever tasted! The atmosphere was cozy, and the staff was friendly. I highly recommend checking it out!"


**Node ID:** db1007e7-6634-4ae3-bf34-282b52c517bf<br>**Similarity:** 0.06225874526574707<br>**Text:** I had the Santa Barbara bowl and it was soooo good! Great coffee, great atmosphere, would for sure come back!<br>

business_id: IDtLPgUrqorrpqSLdfMhZQ
review_stars: 5
biz_name: Helena Avenue Bakery
biz_address: 131 Anacapa St, Ste C
biz_city: Santa Barbara
biz_state: CA
biz_categories: Food, Restaurants, Salad, Coffee & Tea, Breakfast & Brunch, Sandwiches, Bakeries

I had the Santa Barbara bowl and it was soooo good! Great coffee, great atmosphere, would for sure come back!




**Node ID:** fed191b3-98d7-445a-bdb0-b0ea1742ba1d<br>**Similarity:** 0.05665779743414108<br>**Text:** Pricey ( a ham and cheese croissant was $5.50)  as with anywhere in Santa Barbra but very delicious. Cute space ( a bit hard to find ). The green eggs and ham  breakfast biscuit is to die for.  ($10 I think) The Santa Barbra tri tip sandwich was excellent! Cold brew coffee was good. Lots of good choices in the grab and go section, and our salami and pesto sammy was also great.<br>

business_id: IDtLPgUrqorrpqSLdfMhZQ
review_stars: 4
biz_name: Helena Avenue Bakery
biz_address: 131 Anacapa St, Ste C
biz_city: Santa Barbara
biz_state: CA
biz_categories: Food, Restaurants, Salad, Coffee & Tea, Breakfast & Brunch, Sandwiches, Bakeries

Pricey ( a ham and cheese croissant was $5.50)  as with anywhere in Santa Barbra but very delicious. Cute space ( a bit hard to find ). The green eggs and ham  breakfast biscuit is to die for.  ($10 I think) The Santa Barbra tri tip sandwich was excellent! Cold brew coffee was good. Lots of good choices in the grab and go section, and our salami and pesto sammy was also great.




...

business_id: IDtLPgUrqorrpqSLdfMhZQ
review_stars: 5
biz_name: Helena Avenue Bakery
biz_address: 131 Anacapa St, Ste C
biz_city: Santa Barbara
biz_state: CA
biz_categories: Food, Restaurants, Salad, Coffee & Tea, Breakfast & Brunch, Sandwiches, Bakeries

A great little spot for some coffee and treats! Small outdoor area but quaint.




**Node ID:** 4715ea83-d439-4015-821e-6e9ac7f53dc2<br>**Similarity:** 0.023333333333333334<br>**Text:** Right next door to the Santa Barbara Wine Collective, in fact it actually shares the same tables.  We had a fabulous savory croissant.<br>

business_id: IDtLPgUrqorrpqSLdfMhZQ
review_stars: 5
biz_name: Helena Avenue Bakery
biz_address: 131 Anacapa St, Ste C
biz_city: Santa Barbara
biz_state: CA
biz_categories: Food, Restaurants, Salad, Coffee & Tea, Breakfast & Brunch, Sandwiches, Bakeries

Right next door to the Santa Barbara Wine Collective, in fact it actually shares the same tables.  We had a fabulous savory croissant.




**Node ID:** b353731a-4453-4672-a164-e979339b5dde<br>**Similarity:** 0.022983976884686105<br>**Text:** Hot bakery treats abound. Cool vibe central Funky Funk Zone HQ.

Lovely coffees, hipster breakfast treats. Delightful crew, very nice and they do provide good friendly service.

Best breakfast choices include Green Eggs & Ham Biscuit. Yes, lovely with coffee. Avocado toast with an egg is giant size, it's enough for two! 

Bakery treats are endless, fresh breads to go back to the homeland. So fresh and taste so good.<br>

business_id: IDtLPgUrqorrpqSLdfMhZQ
review_stars: 4
biz_name: Helena Avenue Bakery
biz_address: 131 Anacapa St, Ste C
biz_city: Santa Barbara
biz_state: CA
biz_categories: Food, Restaurants, Salad, Coffee & Tea, Breakfast & Brunch, Sandwiches, Bakeries

Hot bakery treats abound. Cool vibe central Funky Funk Zone HQ.

Lovely coffees, hipster breakfast treats. Delightful crew, very nice and they do provide good friendly service.

Best breakfast choices include Green Eggs & Ham Biscuit. Yes, lovely with coffee. Avocado toast with an egg is giant size, it's enough for two! 

Bakery treats are endless, fresh breads to go back to the homeland. So fresh and taste so good.




## Retrieval Evaluation

### Synthetic

In [37]:
from src.run.eval import RetrievalEvaluator

In [38]:
retrieval_evaluator = RetrievalEvaluator()
retrieval_evaluator.generate_synthetic_dataset(cfg, nodes)

2024-08-17 08:08:06.489 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:79 - Loading retrieval_eval_nodes from data/027_auto_retrieval/retrieval_synthetic_eval_dataset.json...
2024-08-17 08:08:06.491 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:85 - Loading existing synthetic retrieval eval dataset at data/027_auto_retrieval/retrieval_synthetic_eval_dataset.json...


In [39]:
retrieval_evaluator.retrieval_eval_dataset.queries

{'08e93640-affd-42d8-9dbc-596629b76bfe': 'What are some restaurants with a lively atmosphere and a diverse menu?',
 '19be6f87-4b10-44e9-89c7-c5dacbfe50ad': 'What are some reliable shipping or delivery services that ensure items are received without damage?',
 'e46d4075-f916-4817-b105-eb63a5fbd584': 'What are some affordable parking options in Midtown Nashville?',
 'ab57ccc0-39cd-4f8e-8ff6-cfba53b0514a': 'What are some businesses or locations that have received negative reviews for their services?',
 '506bd371-6fae-4222-810a-ebe6793a5feb': 'What are some ice cream shops that offer a cozy indoor seating experience during winter?',
 'bd60bae0-c377-4df4-9824-a79fbc777054': 'What are some popular ice cream shops in Nashville known for their waffle cones?',
 '4a6e6914-a631-430f-9ddc-8b15aa84d91c': 'What are some sushi restaurants that offer a la carte options instead of tasting menus?',
 '1b52808b-f784-439a-a4e9-1d6f8e4a3045': 'What are some cafes known for their excellent breakfast sandwich

In [40]:
retrieval_eval_results_df, retrieval_eval_results_full_df = await retrieval_evaluator.aevaluate(cfg, retriever)

Generated queries:
lively atmosphere diverse menu restaurants
"I visited a restaurant with a vibrant ambiance and an extensive menu that offered everything from international cuisine to local favorites. The atmosphere was buzzing, making it a great spot for a night out."
Generated queries:
reliable shipping delivery services items received undamaged
"I recently used QuickShip for my deliveries and was impressed by their careful handling. My fragile items arrived in perfect condition, and their customer service was very responsive."
Generated queries:
affordable parking options midtown nashville
"I recently visited Midtown Nashville and found several affordable parking spots. One great option was the lot near the corner of 21st Avenue, which only charged $5 for the whole day. I also noticed street parking available for just $1 an hour, making it easy to explore the area without breaking the bank."
Generated queries:
negative reviews businesses locations services
Generated queries:
ice c

In [41]:
retrieval_eval_results_df

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,top_50_retrieval_eval,0.666667,0.207793,0.013333,0.666667,0.207793,0.023506


In [42]:
retrieval_eval_results_full_df

,query,expected_ids,retrieved_ids,hit_rate,mrr,precision,recall,ap,ndcg
0,What are some restaurants with a lively atmosp...,[559b7b97-0308-4fc5-96c7-2bb8dfbb1ae0],"[c6a2abe2-1d46-4d9d-a5b6-1fbf7f2ac9b7, a25e715...",1.0,0.100000,0.02,1.0,0.100000,0.022412
1,What are some reliable shipping or delivery se...,[1d921ecb-0a8f-433e-9727-d02f72a8ca02],"[f9159647-1bf3-4e01-9215-ebe2332e77f6, 6ccef97...",1.0,0.142857,0.02,1.0,0.142857,0.025844
2,What are some affordable parking options in Mi...,[6920e739-4736-40bf-8f89-b47ef3837752],"[69401ad3-952e-43e7-986c-242c03d9e4b7, 73fc836...",1.0,0.200000,0.02,1.0,0.200000,0.029994
3,What are some businesses or locations that hav...,[c2aa6622-1b5e-4df5-b3eb-402aebb0dbbf],"[70cadcfb-8f97-47b8-ac0a-052e09b99908, 750120e...",0.0,0.000000,0.00,0.0,0.000000,0.000000
4,What are some ice cream shops that offer a coz...,[2af4563f-207c-47ba-b460-3c512be0ac05],"[bebf2a37-3b12-43fd-b72a-810e1f3b3512, e408bed...",0.0,0.000000,0.00,0.0,0.000000,0.000000
5,What are some popular ice cream shops in Nashv...,[57958b3d-e3e8-406d-a393-ae603046f2ed],"[fdf5cd32-01bc-40a3-9493-8ac05a8e7a18, c2b24b3...",1.0,0.090909,0.02,1.0,0.090909,0.021627
6,What are some sushi restaurants that offer a l...,[26d5e99a-57c7-427c-bacc-923aa03aa4b2],"[f1158a67-f382-4b97-b71b-889fda6d9a19, dd0e12c...",1.0,0.250000,0.02,1.0,0.250000,0.033392
7,What are some cafes known for their excellent ...,[aced15e3-2714-4c31-863c-ace6012b0619],"[a663c8fc-fe97-4aef-b840-2d71cf0e2d36, f6e5ef0...",1.0,0.200000,0.02,1.0,0.200000,0.029994
8,What are some credit card companies known for ...,[25d216e7-96f0-463b-acf5-4aa57d8db9c6],"[7baac717-4e8b-4b45-b353-08385e9b9533, 4fcc829...",1.0,0.035714,0.02,1.0,0.035714,0.015960
9,What are some restaurants that are known for t...,[2e35da2e-3761-44ef-a4d9-5af4671a28db],"[539d0e65-0fde-44d0-af97-d6530a01b83a, 18ee003...",1.0,0.020000,0.02,1.0,0.020000,0.013668


#### Error Analysis

In [43]:
retrieval_eval_irrelevance_df = (
    retrieval_eval_results_full_df
    .loc[lambda df: df['hit_rate'].lt(1)]
    .sort_values(['hit_rate', 'mrr', 'precision', 'recall', 'ap', 'ndcg'])
)
with pd.option_context('display.max_colwidth', 400):
    display(retrieval_eval_irrelevance_df)

,query,expected_ids,retrieved_ids,hit_rate,mrr,precision,recall,ap,ndcg
3,What are some businesses or locations that have received negative reviews for their services?,[c2aa6622-1b5e-4df5-b3eb-402aebb0dbbf],"[70cadcfb-8f97-47b8-ac0a-052e09b99908, 750120eb-4a30-4983-b685-671158a58aaa, 160f6511-9154-4ba6-b201-ed1272900553, 48196d1e-87a5-4d02-8793-97cbaf2c7f1f, 7fb99359-ac3e-4627-8efb-9899ba4e8de8, c72f5696-4bf8-451c-815b-2dcfe10d0172, a9744f24-1b4a-4ac9-ad4f-90534b15e0c3, 1492c139-6555-4be9-943f-57a59271ce75, 31bc008d-6c74-45e4-8387-b1f0418ba3df, 683bd0fb-dd19-4511-9cf1-0bb774f23a9e, 9a62fafd-907e-4...",0.0,0.0,0.0,0.0,0.0,0.0
4,What are some ice cream shops that offer a cozy indoor seating experience during winter?,[2af4563f-207c-47ba-b460-3c512be0ac05],"[bebf2a37-3b12-43fd-b72a-810e1f3b3512, e408bedd-f57e-44a3-8182-ba443d29f002, 2a35e816-0a43-4758-afef-239ab52a78a7, ce40df93-d41c-455c-b8ee-e9b7aaed000e, 4cfad256-2ebc-4d36-bb8e-6737cc2e59f2, 19224ed6-96c4-4b9a-81a4-e4f23e226051, d2017a2b-a68a-4e31-a9d7-2188619ae52d, e234428a-19cd-4c61-8ea2-f0f87fc735ae, 18f95871-bebd-4a38-a5a9-0d965ea4854c, 7b838881-06ec-4e2b-a14e-ec2975c75a9d, 1689632c-a9ba-4...",0.0,0.0,0.0,0.0,0.0,0.0
11,What are some ice cream shops known for their creamy flavors and homemade waffle cones?,[2b7ea064-2c48-4c94-9dbc-dda5b8e28464],"[3605b1a4-b2fe-4b37-8f04-1f6014c344c9, e362dc2b-9669-45a5-8251-b46d7aad832f, a678e1b6-cafd-454f-9921-e7338dd7f730, 39cec30b-bbe8-4881-b1fa-1de8269e18db, c7247c4a-4acd-4a7a-954b-fa3cee7ee6fe, 6c9f26b3-ec46-4df8-81e5-d1aaf7781c15, a12aee1b-ca74-4fc1-9af4-e33d3b0d516c, 337ae1d5-9978-4ffd-9ad9-b64a7c4c360d, 79055e2c-b3ab-44a5-9a96-4db4bb9d7720, 87216a6e-b800-444c-a144-2c7dd8d68c4a, 9e729c71-9e1f-4...",0.0,0.0,0.0,0.0,0.0,0.0
12,What are some popular shopping destinations for clothing and accessories?,[8fb2a28a-3333-495f-85bd-cb4b1fc41e0d],"[32762448-4eec-45ba-956e-64d7bcc93361, cf7933a5-3caa-4a9d-ab48-49330b645553, 11e691ee-7dc2-4bee-80e9-16ed9e07275c, 22c3df3d-16fe-425c-837c-5de6bc3797a0, 50286b5c-c27c-463f-a4d3-1222b4c20e7a, 907b26de-d749-4c5b-b034-56a553c82310, 07d8db8e-8ffc-40ed-b3d4-53adca7b2ca3, 6ab2eb8b-0a50-40a5-bd83-d864862c9ac9, 4d17cff7-9a3d-4a0f-82d1-297b15e0a28e, 3d731991-9801-47d0-a590-3ae9c31855f2, e1be883b-0378-4...",0.0,0.0,0.0,0.0,0.0,0.0
16,What are some popular attractions or activities to enjoy in the area?,[f327755f-f280-4ec0-91bc-1bf2c12a4bef],"[b25f04be-bbdc-42ae-8487-99fb3501745e, e783d169-f785-4087-87c4-b615bfc32e1e, eaa0e7cc-2f19-4408-8c27-eb8130987ce0, 7077579f-f356-47b4-94f5-190def343943, 334a0e53-2837-41ea-97a2-f8791404f553, 548bf227-bf28-4a73-90cd-d6d442300210, 3c118340-149a-4300-876a-fc1f44f5a326, e56e93df-86d4-48ff-a0db-84cde577a87d, d8320667-3d50-463a-b411-3d9263ee519f, 456e23af-f8a4-42f6-aeb6-8327d6c66a6d, 47777f67-264c-4...",0.0,0.0,0.0,0.0,0.0,0.0
17,What are some engaging activities or attractions for children that can create lasting memories?,[c8b68e7b-ccae-4dbb-a92d-640f73c54dca],"[334a0e53-2837-41ea-97a2-f8791404f553, 7077579f-f356-47b4-94f5-190def343943, 3c118340-149a-4300-876a-fc1f44f5a326, 5766bda1-0ac3-46b6-a75b-1d784f35c8e7, d8320667-3d50-463a-b411-3d9263ee519f, e783d169-f785-4087-87c4-b615bfc32e1e, eaa0e7cc-2f19-4408-8c27-eb8130987ce0, 456e23af-f8a4-42f6-aeb6-8327d6c66a6d, 0fbc3b4b-2148-4bcb-bd35-18bc5622a9b5, 370bc7e4-4900-48d4-8a90-9be3f8b307c1, b25f04be-bbdc-4...",0.0,0.0,0.0,0.0,0.0,0.0
21,What are some restaurants known for their excellent customer service and knowledgeable staff?,[1a4388cf-e72b-41ea-8923-0b5fabb459ce],"[388fd603-ef50-4da7-8a31-6b95db0f7922, 6dece51a-59f9-4af8-9e6e-c6ed10221235, 5e09a786-efd6-4fda-8912-d3a0845c4f21, a19768e7-5dd4-4fc5-ad70-65f5d21ac78d, 0e9b2f34-6836-48e5-ad98-28a2343e7d5b, a41eed08-c5b1-4f8e-a3ea-4434b73222ee, c1e12eb1-98db-4487-909a-2f8b163dd110, d99ab5eb-7847-4f79-8b6b-c6588e9a2382, 8135da29-8589-40e4-b271-501e20543e6a, 42b8ad43-d6e2-4622-9aeb-7bbf1eba62df, 0d531b39-c52e-4...",0.0,0.0,0.0,0.0,0.0,0.0
22,What are some popular tasting rooms or wine ba

In [44]:
for i, row in retrieval_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n============Error #{i+1}=============\n\n")
    print(f"Query:\n{row.query}\n")
    print(f"- Expected contexts:\n")
    expected_nodes = docstore.get_nodes(row.expected_ids)
    for node in expected_nodes:
        print(node.get_content(metadata_mode=MetadataMode.LLM))

    print(f"\n- Retrieved contexts:\n")
    retrieved_nodes = docstore.get_nodes(row.retrieved_ids)
    for i, node in enumerate(retrieved_nodes[:10]):
        print(f"+ Context #{i+1}:\n\n{node.get_content(metadata_mode=MetadataMode.LLM)}\n")



============Error #1=============


Query:
What are some businesses or locations that have received negative reviews for their services?

- Expected contexts:

business_id: noByYNtDLQAra9ccqxdfDw
review_stars: 1
biz_name: H&M
biz_address: 827-833 State St
biz_city: Santa Barbara
biz_state: CA
biz_categories: Women's Clothing, Accessories, Children's Clothing, Men's Clothing, Adult, Shopping, Fashion

Awful.

- Retrieved contexts:

+ Context #1:

business_id: 7PDi_iyik3jraDAzWwwR4Q
review_stars: 1
biz_name: Chase JP Morgan Bank Credit Card Services
biz_address: 
biz_city: Wilmington
biz_state: DE
biz_categories: Banks & Credit Unions, Financial Services

That is NOT how you keep clients happy! But judging by all of negative reviews here, I doubt someone would read this or care enough about the reviews to do anything about it. 
You have been warned! Stay away from Chase!

+ Context #2:

business_id: B2qqjCl_BNNnXvWWfpFiLA
review_stars: 1
biz_name: Pottery Barn Kids
biz_address: 4011 Hi

In [45]:
def check_retrieval_results(query):
    retrieval_results = retriever.retrieve(query)
    for node in retrieval_results:
        display_source_node(node, source_length=1000)
        print(node.get_content(metadata_mode=MetadataMode.EMBED))
        print("\n")
    return retrieval_results

query = "What are some good grocery stores in Nashville that offer curated items and unique local products?"
check_retrieval_results(query)

Generated queries:
grocery stores nashville curated items local products
"I recently visited a grocery store in Nashville that had an amazing selection of curated items and unique local products. The atmosphere was inviting, and I found some fantastic local jams and artisanal cheeses that I couldn't resist!"


**Node ID:** 6920e739-4736-40bf-8f89-b47ef3837752<br>**Similarity:** 0.044579052788008014<br>**Text:** Interior is gorgeous. Very clean and modern. It'll be great for a lot of Nashville and tourist Instagrammers to get their pose on. 

As far as groceries, lots of curated items from both Nashville and beyond. They have McClures pickles... being from Michigan I got super pumped about those. They also carry Jenis Ice Cream pints which are down right dangerous. Great selection of coffee, sweets, cheeses, charcuterie and wine.

The only reason it's not a full 5 is the parking situation. All the lots in midtown are super expensive or you need change for meters (it's 2017 I don't carry anything that's monetary unless it's a card). If you do meters, it's $1.50 per hour in that area.  Paid lots are about $8 an hour.<br>

business_id: lk9IwjZXqUMqqOhM774DtQ
review_stars: 4
biz_name: Caviar & Bananas
biz_address: 2031 Broadway
biz_city: Nashville
biz_state: TN
biz_categories: Coffee & Tea, Restaurants, Wine Bars, Bars, Nightlife, American (Traditional), Event Planning & Services, Food, Caterers, Breakfast & Brunch, Cafes, Diners

Interior is gorgeous. Very clean and modern. It'll be great for a lot of Nashville and tourist Instagrammers to get their pose on. 

As far as groceries, lots of curated items from both Nashville and beyond. They have McClures pickles... being from Michigan I got super pumped about those. They also carry Jenis Ice Cream pints which are down right dangerous. Great selection of coffee, sweets, cheeses, charcuterie and wine.

The only reason it's not a full 5 is the parking situation. All the lots in midtown are super expensive or you need change for meters (it's 2017 I don't carry anything that's monetary unless it's a card). If you do meters, it's $1.50 per hour in that area.  Pa

**Node ID:** 54712e50-9680-40e3-82af-ce6d69afed94<br>**Similarity:** 0.043030036562225205<br>**Text:** This is a very interesting spot for specialty items and local items like that delicious Nashville fountain of juice. This is a great place to grab coffee, dessert or a salad / sandwich for lunch.<br>

business_id: lk9IwjZXqUMqqOhM774DtQ
review_stars: 4
biz_name: Caviar & Bananas
biz_address: 2031 Broadway
biz_city: Nashville
biz_state: TN
biz_categories: Coffee & Tea, Restaurants, Wine Bars, Bars, Nightlife, American (Traditional), Event Planning & Services, Food, Caterers, Breakfast & Brunch, Cafes, Diners

This is a very interesting spot for specialty items and local items like that delicious Nashville fountain of juice. This is a great place to grab coffee, dessert or a salad / sandwich for lunch.




...

**Node ID:** d048388c-cb1e-4bef-b7f4-2231b9fee559<br>**Similarity:** 0.013888888888888888<br>**Text:** Great ice cream with a wonderful assortment of flavors.  Anytime I visit downtown Nashville, a stop here is almost a requirement.<br>

business_id: oaboaRBUgGjbo2kfUIKDLQ
review_stars: 4
biz_name: Mike's Ice Cream
biz_address: 129 2nd Ave N
biz_city: Nashville
biz_state: TN
biz_categories: Ice Cream & Frozen Yogurt, Coffee & Tea, Restaurants, Sandwiches, Food

Great ice cream with a wonderful assortment of flavors.  Anytime I visit downtown Nashville, a stop here is almost a requirement.




**Node ID:** 2e42594d-0f83-43d1-ba82-caee06af097b<br>**Similarity:** 0.0136986301369863<br>**Text:** The food was decent once we got it but honestly, not all that great considering the hassle you have to go through to get it and eat. There are a million great places to eat in Nashville...I'd put this low on the list.<br>

business_id: lk9IwjZXqUMqqOhM774DtQ
review_stars: 2
biz_name: Caviar & Bananas
biz_address: 2031 Broadway
biz_city: Nashville
biz_state: TN
biz_categories: Coffee & Tea, Restaurants, Wine Bars, Bars, Nightlife, American (Traditional), Event Planning & Services, Food, Caterers, Breakfast & Brunch, Cafes, Diners

The food was decent once we got it but honestly, not all that great considering the hassle you have to go through to get it and eat. There are a million great places to eat in Nashville...I'd put this low on the list.




[NodeWithScore(node=TextNode(id_='6920e739-4736-40bf-8f89-b47ef3837752', embedding=None, metadata={'review_id': 'VZMjK-Moy-fN08mR2m9JUg', 'user_id': 'U5hfpmnqW65_u10C1i06Dw', 'business_id': 'lk9IwjZXqUMqqOhM774DtQ', 'review_stars': 4, 'useful': 4, 'funny': 1, 'cool': 0, 'date': '2017-06-23T12:16:51', 'biz_name': 'Caviar & Bananas', 'biz_address': '2031 Broadway', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37203', 'biz_latitude': 36.1483712, 'biz_longitude': -86.7988947, 'biz_stars': 3.5, 'biz_review_count': 159, 'biz_is_open': 0, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'beer_and_wine\'", \'Ambience\': "{\'touristy\': False, \'hipster\': False, \'romantic\': False, \'divey\': False, \'intimate\': False, \'trendy\': True, \'upscale\': False, \'classy\': True, \'casual\': True}", \'BYOB\': None, \'BestNights\': "{\'monday\': False, \'tuesday\': False, \'friday\': False, \'wednesday\': False, \'thursday\': False, \'sunday\': False, \'saturday\':

# Response

In [46]:
from llama_index.core import get_response_synthesizer
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker
from llama_index.core.response_synthesizers.type import ResponseMode

from src.features.append_reference.custom_query_engine import ManualAppendReferenceQueryEngine
from src.features.synthesize_recommendation.custom_tree_summarize import CUSTOM_TREE_SUMMARIZE_PROMPT_SEL

In [47]:
node_postprocessors = []

if cfg.retrieval_cfg.retrieval_similarity_cutoff is not None:
    node_postprocessors.append(SimilarityPostprocessor(similarity_cutoff=cfg.retrieval_cfg.retrieval_similarity_cutoff))

reranker = FlagEmbeddingReranker(model=cfg.retrieval_cfg.rerank_model_name, top_n=cfg.retrieval_cfg.rerank_top_k, use_fp16=True)
node_postprocessors.append(reranker)

response_synthesizer = get_response_synthesizer(
    response_mode=ResponseMode.TREE_SUMMARIZE,
    summary_template=CUSTOM_TREE_SUMMARIZE_PROMPT_SEL
)
query_engine = ManualAppendReferenceQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=node_postprocessors,
)

## Test Query Engine

In [48]:
from llama_index.core.response.notebook_utils import (
    display_source_node,
    display_response,
)

In [49]:
# query = "What are some gastropubs in Indianapolis that are known for their cocktails?"
query = "What are some places to enjoy cold brew coffee"
response = query_engine.query(query)
display_response(response, source_length=1000, show_source=True, show_metadata=True, show_source_metadata=True)

Generated queries:
cold brew coffee places
"I recently visited a café that specializes in cold brew coffee, and I was blown away by their unique flavors and smooth texture. The atmosphere was cozy, making it a perfect spot to relax and enjoy a refreshing drink."


**`Final Response:`** Based on your interest in enjoying cold brew coffee, here are several recommended places ranked by relevance:

### 1. **Helena Avenue Bakery**
   - **Location:** 131 Anacapa St, Ste C, Santa Barbara, CA
   - **Review Stars:** 4-5
   - **Strengths:** Helena Avenue Bakery is highly praised for its cold brew coffee, which has received multiple mentions in reviews. The bakery offers a cozy atmosphere with a cute space and a lovely patio to enjoy your drinks. In addition to cold brew, they serve a variety of delicious breakfast items and pastries, making it a great spot for a leisurely morning or brunch. The quality of their coffee, including cappuccinos and cold brews, is noted to be impressive.
   - **Ideal For:** Those looking for a combination of great cold brew coffee and a delightful breakfast or brunch experience in a charming setting.

### 2. **Mike's Ice Cream**
   - **Location:** 129 2nd Ave N, Nashville, TN
   - **Review Stars:** 4-5
   - **Strengths:** While primarily known for its ice cream, Mike's Ice Cream also offers a variety of coffee options, including cold brew. The reviews highlight the friendly staff and the quality of their coffee, making it a good alternative if you're in the mood for something sweet alongside your drink. The atmosphere is casual and fun, perfect for a quick stop or a relaxed hangout.
   - **Ideal For:** Those who want to enjoy cold brew coffee in a laid-back environment, possibly paired with a delicious ice cream treat.

### 3. **Wawa**
   - **Location:** 3604 Chestnut St, Philadelphia, PA
   - **Review Stars:** 4
   - **Strengths:** Wawa is known for its convenience and variety of food and drink options, including iced coffee. The reviews mention the iced coffee as being particularly good, and the ability to customize your drink adds to the appeal. While it may not have the same ambiance as a dedicated coffee shop, it is a reliable option for a quick and refreshing cold brew.
   - **Ideal For:** Those looking for a quick and convenient stop for cold brew coffee, especially if you're on the go.

### 4. **Roast Coffeehouse and Wine Bar**
   - **Location:** 10359 104 Street NW, Edmonton, AB
   - **Review Stars:** 3
   - **Strengths:** Roast Coffeehouse offers a variety of coffee options, including flavored lattes and potentially cold brew. However, the reviews indicate mixed experiences with the quality of their coffee, particularly espresso. While it has a nice atmosphere and supports local craft brews and wines, it may not be the best choice if you're specifically seeking high-quality cold brew.
   - **Ideal For:** Those who enjoy a variety of coffee drinks and a casual atmosphere, but may not be the best choice for cold brew enthusiasts seeking top-notch quality.

### Summary Comparison:
- **Helena Avenue Bakery** stands out as the top choice for a dedicated cold brew experience paired with excellent food in a charming setting.
- **Mike's Ice Cream** offers a fun twist by combining cold brew with ice cream, making it a great spot for a casual outing.
- **Wawa** is perfect for those needing a quick and convenient option without the need for a sit-down experience.
- **Roast Coffeehouse and Wine Bar** may appeal to those looking for a variety of drinks but might not satisfy a cold brew aficionado due to mixed reviews on coffee quality.

Overall, if your primary goal is to enjoy high-quality cold brew coffee, **Helena Avenue Bakery** is the best option, followed closely by **Mike's Ice Cream** for a more casual experience.


#### Referenced Paragraphs
**Helena Avenue Bakery**

> Pricey ( a ham and cheese croissant was $5.50)  as with anywhere in Santa Barbra but very delicious. Cute space ( a bit hard to find ). The green eggs and ham  breakfast biscuit is to die for.  ($10 I think) The Santa Barbra tri tip sandwich was excellent! Cold brew coffee was good. Lots of good choices in the grab and go section, and our salami and pesto sammy was also great.  (Review ID: <REVIEW_ID>Q3fPo_x6xKxafAzy1hFITg</REVIEW_ID>)

> This review is for the delicious breakfast and coffee. The Green Breakfast Bowl and the Biscuit sandwich were both perfectly cooked, fresh, flavorful. We were really impressed with the quality of the cappuccino and the cold brew coffee. The donut was hearty-tasting and clearly a higher quality of bread compared to your typical donut. We came in at around ten on a Saturday and there were plenty of spots to sit even though it is clearly a popular place to be.  (Review ID: <REVIEW_ID>4sGHeBRdn36yOX3KszhzCA</REVIEW_ID>)

> We got the green eggs and ham, savory scone, cold brew, and mimosas! Their patio is delicious to sit at and enjoy the food! (Review ID: <REVIEW_ID>ybgnv1EGL7U4urhPmPuf4w</REVIEW_ID>)

> Cold brew was sooo good as were the sandwiches we got. Will most definitely be back. (Review ID: <REVIEW_ID>-6iyNp8Vs9kF0IRGXZvsHA</REVIEW_ID>)

> Barista was fast at least. Good latte and cold brew. I should have just ordered a pastry though. (Review ID: <REVIEW_ID>LuLzCVNRkAPjVaRVA__V2A</REVIEW_ID>)

**Caviar & Bananas**

> Need a bottle of wine? Check!
Need a cold brew coffee with just a splash of almond milk? Check!
Need eggs? Check!
Need a gift basket? Check!
Need a frozen Rose? (Oh didn't know I needed that but I DO!) check! 
Neeeeeed that chocolate? Check!
Need a place for an informal meet up. Check! 
Need a fast bar meal 8:00 pm? Check!

 (Review ID: <REVIEW_ID>TO650uj_N9YlOUFjk9K2eQ</REVIEW_ID>)

**Mike's Ice Cream**

> If it's cold, come grab a coffee instead. They offer all the great styles and flavors of coffee as any other coffee house you'll find. (Review ID: <REVIEW_ID>GHAY9_Kpk-1hcq4Ehfmu1w</REVIEW_ID>)

> I am a big fan of diverse ice cream flavors, and this place definitely has that. If you are cold and not necessarily feeling a frosty treat, they also have different hot chocolate flavors from around the world-the Italian hot chocolate is amazing, by the way! The staff were friendly and willing to go the extra mile to ensure that we were enjoying our coffee.  (Review ID: <REVIEW_ID>S2hL5Cnyzs2ck5xiH8WKGQ</REVIEW_ID>)

**Wawa**

> The sandwiches are really the central offering here, but I've made a few other deadly discoveries: the iced coffee and milkshakes. I'm used to having a barista prepare my iced coffee, as opposed to a massive grumbling chunk of machinery, but this stuff is GOOD. It's pre-sweetened, pre-lactated (poor word choice? I think yes), super yummy, and delightfully refreshing!! It somehow strikes the perfect level of coldness (and look ma, no ice cubes!!). 

You can also "make" your own milkshake by taking a cup (basically filled with ice cream) and sticking it in another machine, which will then proceed to slash away at the ice cream with a couple of blades until it reaches your desired thickness. I always complain about runny milkshakes, but should you select the "thick milkshake" option (yeah baby), you won't be having that problem here - trust. (Review ID: <REVIEW_ID>z7PbqtG11XZ9ircBOV0THQ</REVIEW_ID>)

**Roast Coffeehouse and Wine Bar**

> ...I think they briefly had a caramel-bacon latte which sounds really interesting and I enjoyed their salted caramel latte.  They have a collection of craft beers and wines mainly from AB and the Okanagan region.  It's interesting and I'm glad they support local craft brews and wines, though at least on the beer front, these regions are not known for the best craft brews in Canada.

Why the 3 stars? Many might consider me a coffee snob when I say this, but I actually think that for $5, it's only fair that the quality of the actual espresso be far better than something you would get at a chain coffee shop.  This, however, they lack in.  The staff seem to know very little about actual coffee and how to make a good espresso and this disappoints me a little.  It's got a bit of a yuppie feel to it without delivering a quality product.  

I am not sure if I will go back because I know of places that make better coffee. There are many in Edmonton now (Elm, Transcend, Leva, Credo, Sugar Bowl, da ...

---

**`Source Node 1/10`**

**Node ID:** fed191b3-98d7-445a-bdb0-b0ea1742ba1d<br>**Similarity:** -1.5986328125<br>**Text:** Pricey ( a ham and cheese croissant was $5.50)  as with anywhere in Santa Barbra but very delicious. Cute space ( a bit hard to find ). The green eggs and ham  breakfast biscuit is to die for.  ($10 I think) The Santa Barbra tri tip sandwich was excellent! Cold brew coffee was good. Lots of good choices in the grab and go section, and our salami and pesto sammy was also great.<br>**Metadata:** {'review_id': 'Q3fPo_x6xKxafAzy1hFITg', 'user_id': 'ha2Lv7WnxvdYnRsqwL1apQ', 'business_id': 'IDtLPgUrqorrpqSLdfMhZQ', 'review_stars': 4, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2018-07-17T00:20:41', 'biz_name': 'Helena Avenue Bakery', 'biz_address': '131 Anacapa St, Ste C', 'biz_city': 'Santa Barbara', 'biz_state': 'CA', 'biz_postal_code': '93101', 'biz_latitude': 34.4144445, 'biz_longitude': -119.6906718, 'biz_stars': 4.0, 'biz_review_count': 389, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'touristy\': False, \'hipster\': True, \'romantic\': False, \'divey\': False, \'intimate\': False, \'trendy\': True, \'upscale\': False, \'classy\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'True\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'True\', \'DriveThru\': None, \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': "{\'dessert\': False, \'latenight\': False, \'lunch\': True, \'dinner\': False, \'brunch\': True, \'breakfast\': True}", \'HappyHour\': \'False\', \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'False\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': \'True\', \'WiFi\': "u\'no\'"}', 'biz_categories': 'Food, Restaurants, Salad, Coffee & Tea, Breakfast & Brunch, Sandwiches, Bakeries'}<br>

...

**Node ID:** 72fc9683-c951-4e63-bb82-68e7a4f4052c<br>**Similarity:** -4.82421875<br>**Text:** The sandwiches are really the central offering here, but I've made a few other deadly discoveries: the iced coffee and milkshakes. I'm used to having a barista prepare my iced coffee, as opposed to a massive grumbling chunk of machinery, but this stuff is GOOD. It's pre-sweetened, pre-lactated (poor word choice? I think yes), super yummy, and delightfully refreshing!! It somehow strikes the perfect level of coldness (and look ma, no ice cubes!!). 

You can also "make" your own milkshake by taking a cup (basically filled with ice cream) and sticking it in another machine, which will then proceed to slash away at the ice cream with a couple of blades until it reaches your desired thickness. I always complain about runny milkshakes, but should you select the "thick milkshake" option (yeah baby), you won't be having that problem here - trust.<br>**Metadata:** {'review_id': 'z7PbqtG11XZ9ircBOV0THQ', 'user_id': '76dzq9rVZIMl_oEvAVickg', 'business_id': 'ppFCk9aQkM338Rgwpl2F5A', 'review_stars': 4, 'useful': 0, 'funny': 1, 'cool': 0, 'date': '2011-05-19T02:54:32', 'biz_name': 'Wawa', 'biz_address': '3604 Chestnut St', 'biz_city': 'Philadelphia', 'biz_state': 'PA', 'biz_postal_code': '19104', 'biz_latitude': 39.954573, 'biz_longitude': -75.1948936, 'biz_stars': 3.0, 'biz_review_count': 56, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'classy\': False, \'hipster\': False, \'divey\': False, \'touristy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': \'False\', \'Caters\': \'False\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': None, \'DriveThru\': None, \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': \'True\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'False\', \'RestaurantsAttire\': "u\'casual\'", \'RestaurantsDelivery\': \'True\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'1\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "u\'free\'"}', 'biz_categories': 'Restaurants, Automotive, Delis, Gas Stations, Food, Coffee & Tea, Sandwiches, Convenience Stores'}<br>

---

**`Source Node 10/10`**

**Node ID:** 8cf98f58-64bc-430b-a14b-3e354ff6b562<br>**Similarity:** -5.0390625<br>**Text:** I think they briefly had a caramel-bacon latte which sounds really interesting and I enjoyed their salted caramel latte.  They have a collection of craft beers and wines mainly from AB and the Okanagan region.  It's interesting and I'm glad they support local craft brews and wines, though at least on the beer front, these regions are not known for the best craft brews in Canada.

Why the 3 stars? Many might consider me a coffee snob when I say this, but I actually think that for $5, it's only fair that the quality of the actual espresso be far better than something you would get at a chain coffee shop.  This, however, they lack in.  The staff seem to know very little about actual coffee and how to make a good espresso and this disappoints me a little.  It's got a bit of a yuppie feel to it without delivering a quality product.  

I am not sure if I will go back because I know of places that make better coffee. There are many in Edmonton now (Elm, Transcend, Leva, Credo, Sugar Bowl, ...<br>**Metadata:** {'review_id': 'TjAHWkX50f4VlWEuY9l2dw', 'user_id': 'sVr6_v-lwixu3VREOO9XpA', 'business_id': 'WKMJwqnfZKsAae75RMP6jA', 'review_stars': 3, 'useful': 5, 'funny': 0, 'cool': 1, 'date': '2012-12-18T03:21:27', 'biz_name': 'Roast Coffeehouse and Wine Bar', 'biz_address': '10359 104 Street NW', 'biz_city': 'Edmonton', 'biz_state': 'AB', 'biz_postal_code': 'T5J 1B9', 'biz_latitude': 53.5460453, 'biz_longitude': -113.4991693, 'biz_stars': 4.0, 'biz_review_count': 40, 'biz_is_open': 0, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'beer_and_wine\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'classy\': False, \'hipster\': True, \'touristy\': False, \'trendy\': True, \'upscale\': False, \'casual\': False}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': None, \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': None, \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'True\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': None, \'DriveThru\': None, \'GoodForDancing\': None, \'GoodForKids\': \'False\', \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'False\', \'RestaurantsAttire\': "u\'casual\'", \'RestaurantsDelivery\': \'False\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "u\'free\'"}', 'biz_categories': 'Coffee & Tea, Food, Cafes, Bars, Wine Bars, Restaurants, Nightlife'}<br>

{'fed191b3-98d7-445a-bdb0-b0ea1742ba1d': {'review_id': 'Q3fPo_x6xKxafAzy1hFITg',
  'user_id': 'ha2Lv7WnxvdYnRsqwL1apQ',
  'business_id': 'IDtLPgUrqorrpqSLdfMhZQ',
  'review_stars': 4,
  'useful': 0,
  'funny': 0,
  'cool': 0,
  'date': '2018-07-17T00:20:41',
  'biz_name': 'Helena Avenue Bakery',
  'biz_address': '131 Anacapa St, Ste C',
  'biz_city': 'Santa Barbara',
  'biz_state': 'CA',
  'biz_postal_code': '93101',
  'biz_latitude': 34.4144445,
  'biz_longitude': -119.6906718,
  'biz_stars': 4.0,
  'biz_review_count': 389,
  'biz_is_open': 1,
  'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'touristy\': False, \'hipster\': True, \'romantic\': False, \'divey\': False, \'intimate\': False, \'trendy\': True, \'upscale\': False, \'classy\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': Fals

## Response Evaluation

In [50]:
from src.run.eval import ResponseEvaluator

In [51]:
response_evaluator = ResponseEvaluator()

### Synthetic

In [52]:
response_eval_documents, response_synthetic_eval_dataset = response_evaluator.generate_synthetic_dataset(cfg, documents)

2024-08-17 08:09:43.375 | INFO     | src.run.eval.response:generate_synthetic_dataset:40 - Sampling 30 documents for response evaluation...
2024-08-17 08:09:43.382 | INFO     | src.run.eval.response:generate_synthetic_dataset:60 - Creating new response eval dataset at data/027_auto_retrieval/response_synthetic_eval_dataset.json...
2024-08-17 08:09:43.383 | INFO     | src.run.eval.response:generate_synthetic_dataset:63 - Creating synthetic response eval dataset...


Parsing nodes:   0%|          | 0/30 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.22s/it]
2024-08-17 08:10:31.709 | INFO     | src.run.eval.response:generate_synthetic_dataset:86 - Persisting synthetic response eval dataset at data/027_auto_retrieval/response_synthetic_eval_dataset.json...


In [53]:
response_synthetic_eval_prediction_dataset = await response_synthetic_eval_dataset.amake_predictions_with(
    predictor=query_engine, batch_size=cfg.batch_size, show_progress=True
)

Batch processing of predictions:   0%|                                                                                                                                                                                                | 0/1 [00:00<?, ?it/s]

Generated queries:
highly-rated Japanese restaurants Nashville variety dishes
"I recently dined at a fantastic Japanese restaurant in Nashville that had an extensive menu. The sushi was fresh, and they also offered delicious ramen and tempura. Definitely a place to explore for anyone craving Japanese cuisine!"


Batch processing of predictions:   0%|                                                                                                                                                                                                | 0/1 [00:00<?, ?it/s]

Generated queries:
Italian restaurants excellent customer service Reno
"I recently dined at Bella Italia in Reno, and the service was outstanding! The staff was attentive and friendly, making our experience truly enjoyable. Highly recommend for anyone looking for great Italian food and top-notch service!"


Batch processing of predictions:   0%|                                                                                                                                                                                                | 0/1 [00:00<?, ?it/s]

...

Batch processing of predictions:   0%|                                                                                                                                                                                                | 0/1 [00:00<?, ?it/s]

Generated queries:
breweries craft beers indianapolis
"I visited Sun King Brewery in Indianapolis, and they had an amazing selection of craft beers, including IPAs and stouts. The atmosphere was great, and the staff was very knowledgeable about their offerings."


Batch processing of predictions:   0%|                                                                                                                                                                                                | 0/1 [00:00<?, ?it/s]

Generated queries:
highly-rated wine bars new orleans great atmosphere
"I visited a wine bar in New Orleans that had an amazing ambiance and an extensive selection of wines. The atmosphere was perfect for a night out, and the staff was knowledgeable about their offerings."


Batch processing of predictions: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.13s/it]


In [54]:
response_synthetic_mean_scores_df, response_synthetic_deep_eval_df = response_evaluator.evaluate_labelled_rag_dataset(
    response_synthetic_eval_dataset,
    response_synthetic_eval_prediction_dataset,
    dataset_name="synthetic",
    judge_model=cfg.eval_cfg.response_eval_llm_model,
    cache_dp=cfg.notebook_cache_dp
)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [03:05<00:00,  6.18s/it]


In [55]:
response_synthetic_mean_scores_df

rag,base_rag
metrics,
mean_correctness_score,4.466667
mean_relevancy_score,0.933333
mean_faithfulness_score,0.966667
mean_context_relevancy_score,0.691667


In [56]:
response_synthetic_deep_eval_df

,query,answer,relevancy_score,correctness_score,faithfulness_score,context_relevancy_score,contexts
0,What are some highly-rated Japanese restaurant...,"\nBased on the provided information, the top r...",1.0,5.0,1.0,0.750,[One of my favorite restaurants in Nashville. ...
1,- What are some Italian restaurants known for ...,\nBased on the provided reviews for Romano's M...,1.0,3.0,1.0,0.625,"[Service was excellent. , Chicken parm was goo..."
2,What are some highly-rated wine bars in New Or...,"\nBased on the provided information, here are ...",1.0,5.0,1.0,0.750,"[Great service, great wine and delicious food...."
3,- What are some highly-rated wine bars in New ...,"\nBased on the provided information, the top r...",1.0,5.0,1.0,0.750,[The only thing I can think to say I would hav...
4,- What are some ice cream shops in Nashville t...,\nBased on your query for ice cream shops in N...,1.0,5.0,1.0,0.750,[While walking around downtown I kept seeing p...
5,What are some coffeehouses in Edmonton that of...,\nBased on your query for coffeehouses in Edmo...,1.0,5.0,1.0,0.750,[I went there yesterday afternoon for the firs...
6,- What are some highly-rated sports bars that ...,\nBased on your query for highly-rated sports ...,1.0,5.0,1.0,0.625,[I highly recommend checking out Top Shelf Spo...
7,- What are some cocktail bars in Philadelphia ...,\nBased on your query for cocktail bars in Phi...,1.0,5.0,1.0,0.750,[Greta happy hour; 1/2 priced all drinks excep...
8,- What are some sports bars in Indianapolis th...,"\nBased on the provided information, here are ...",1.0,5.0,1.0,0.750,[The best part obviously is the scenery and fr...
9,What are some popular wine bars in New Orleans...,"\nBased on the provided information, here are ...",1.0,4.0,1.0,0.750,[Cool wine bar located in the CBD. They have a...


#### Error Analysis

In [57]:
response_synthetic_eval_dataset_dict = dict()
for example in response_synthetic_eval_dataset.examples:
    response_synthetic_eval_dataset_dict[example.query] = {
        "reference_answer": example.reference_answer,
        "reference_contexts": example.reference_contexts,
    }

In [58]:
response_synthetic_deep_eval_df.sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score', 'context_relevancy_score'])

,query,answer,relevancy_score,correctness_score,faithfulness_score,context_relevancy_score,contexts
23,- What are some pizza places known for their q...,\nBased on the provided reviews for Domino's P...,0.0,2.0,1.0,0.250,[Dominoes is getting better lately about getti...
14,- What are some sports bars in Indianapolis th...,\nBased on the provided reviews for Twin Peaks...,0.0,2.0,1.0,0.625,[Drinks are fine. The service completely varie...
1,- What are some Italian restaurants known for ...,\nBased on the provided reviews for Romano's M...,1.0,3.0,1.0,0.625,"[Service was excellent. , Chicken parm was goo..."
10,What are some popular ice cream shops in Nashv...,\nBased on your query for popular ice cream sh...,1.0,3.0,1.0,0.750,[While walking around downtown I kept seeing p...
24,- What are some highly-rated cafes in Nashvill...,"\nBased on the provided information, here are ...",1.0,4.0,0.0,0.625,[We need to come back so we can try more. Gre...
22,- What are some gastropubs in Indianapolis tha...,"\nBased on the provided information, here are ...",1.0,4.0,1.0,0.625,[Def not as good as Chumley's. Probably not co...
25,- What are some mobile phone stores known for ...,\nBased on the provided reviews for the T-Mobi...,1.0,4.0,1.0,0.625,[I usually love going to this t-mobile. The re...
9,What are some popular wine bars in New Orleans...,"\nBased on the provided information, here are ...",1.0,4.0,1.0,0.750,[Cool wine bar located in the CBD. They have a...
20,- What are some sushi bars in Philadelphia wit...,"\nBased on the provided reviews, here are the ...",1.0,4.0,1.0,0.750,[4.0 - 4.5 star Review. I compare all sushi re...
21,- What are some popular brunch spots in New Or...,"\nBased on the provided information, here are ...",1.0,4.0,1.0,0.750,[Brunch in the Big Easy! There are so many sp...


In [59]:
CONTEXT_RELEVANCE_SCORE_THRESHOLD = 0.5

response_synthetic_eval_irrelevance_df = (
    response_synthetic_deep_eval_df
    .loc[lambda df: df['context_relevancy_score'].le(CONTEXT_RELEVANCE_SCORE_THRESHOLD)]
    .sort_values(['context_relevancy_score', 'relevancy_score', 'correctness_score', 'faithfulness_score'])
)

with pd.option_context('display.max_colwidth', 400):
    display(response_synthetic_eval_irrelevance_df)

,query,answer,relevancy_score,correctness_score,faithfulness_score,context_relevancy_score,contexts
23,"- What are some pizza places known for their quality and flavor in White House, TN?","\nBased on the provided reviews for Domino's Pizza in White House, TN, here are the recommendations ranked by relevance to the user's query about pizza places known for their quality and flavor:\n\n### 1. **Domino's Pizza**\n - **Address:** 3001 Highway 31 W, White House, TN\n - **Rating Summary:** The reviews for Domino's Pizza are mixed, with some customers praising the quality and servi...",0.0,2.0,1.0,0.25,"[Dominoes is getting better lately about getting out your order pretty quickly while it's still hot. We order about once a week and they have always gotten our order right. , Great pizzas and great service. Love the order on-line or order by app feature., Undercook and overpaid! Got a pizza and it wasn't even fully cook, pretty gross. We were between Little Caesar and Dominos, thought we'd pa..."


In [60]:
for i, row in response_synthetic_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n==============Error #{i+1}===============\n\n")
    print(f"Query:\n{row.query}\n")
    contexts = '\n\n'.join(row.contexts)
    print(f"Context:\n{contexts}\n")
    print(f"Answer:\n{row.answer}\n----\n")
    expected = response_synthetic_eval_dataset_dict.get(row.query)
    if not expected:
        logger.error(f"Could not find query {row.query} in synthetic_response_eval_dataset_dict!")
        continue
    expected_answer = expected['reference_answer']
    print(f"Expected Answer:\n{expected_answer}\n")
    expected_contexts = expected['reference_contexts']
    print(f"Expected Contexts:\n{expected_contexts}\n")



==============Error #1===============


Query:
- What are some pizza places known for their quality and flavor in White House, TN?

Context:
Dominoes is getting better lately about getting out your order pretty quickly while it's still hot. We order about once a week and they have always gotten our order right. 

Great pizzas and great service.  Love the order on-line or order by app feature.

Undercook and overpaid! Got a pizza and it wasn't even fully cook, pretty gross. We were between Little Caesar and Dominos, thought we'd pay a little more for a "better" pizza WRONG. 

OMG!  It was around $26 for an XL pepperoni pizza, nothing else on it.  I went to the WH pizza shop on 31 and got one loaded with toppings and it was about the same price and it was delicious!  Dominos is horrible & I will never go back, skip this place, save your $$.

Domino's keep being you and I'll keep patronizing your fine establishment.

So this is the pizza joint my son and I hit up. Here's what  they do r

In [61]:
# query = "- What are some pizza places known for their quality and flavor in White House, TN?"
query = "What are some sports bars in Indianapolis that offer a better dining experience than Twin Peaks?	"
response = query_engine.query(query)
display_response(response, source_length=1000, show_source=True, show_metadata=True, show_source_metadata=True)

Generated queries:
sports bars dining experience indianapolis better than Twin Peaks
"I visited a sports bar in Indianapolis called The District Tap, and I was really impressed with their menu and atmosphere. The food was delicious, and they had a great selection of craft beers, making it a perfect spot for a fun night out."


**`Final Response:`** Based on the provided information, here are some recommendations for sports bars in Indianapolis that may offer a better dining experience than Twin Peaks, ranked by relevance:

### 1. **Hooters**
   - **Strengths**: Hooters is a well-known chain that offers a similar atmosphere to Twin Peaks but is often considered a more established option. It provides a casual dining experience with a focus on wings and beer, and it has a reputation for good service. Many reviews suggest that while Hooters has its charm, it may not match the quality of food and service found at Twin Peaks.
   - **Comparison**: Hooters is a direct competitor to Twin Peaks, but some reviews indicate that Twin Peaks may have better food quality and customer service. However, Hooters is a familiar choice for those looking for a traditional sports bar experience.

### 2. **Buffalo Wild Wings**
   - **Strengths**: Known for its extensive menu of wings, beers, and sports viewing options, Buffalo Wild Wings offers a lively atmosphere with numerous TVs for game-watching. The food is generally well-reviewed, especially the wings, and the service is typically friendly.
   - **Comparison**: Buffalo Wild Wings may provide a more diverse menu and a more family-friendly environment compared to Twin Peaks. While Twin Peaks has a unique theme, Buffalo Wild Wings is often praised for its food quality and variety, making it a strong alternative.

### 3. **The Tilted Kilt**
   - **Strengths**: Similar to Twin Peaks, The Tilted Kilt features a sports bar atmosphere with servers in themed attire. It offers a menu that includes pub fare and a variety of beers. The ambiance is often described as fun and lively, making it a great place to watch sports.
   - **Comparison**: The Tilted Kilt is comparable to Twin Peaks in terms of theme and atmosphere, but it may offer a different menu and experience. Some patrons may prefer the food and service at The Tilted Kilt over Twin Peaks, making it a viable alternative.

### 4. **The Foxhole**
   - **Strengths**: This local sports bar is known for its welcoming atmosphere and good food. It offers a variety of American fare and has a reputation for being a great place to watch games with friends. The service is often highlighted as friendly and attentive.
   - **Comparison**: The Foxhole may not have the same themed atmosphere as Twin Peaks, but it is praised for its food quality and service. It could be a better option for those looking for a more laid-back dining experience without the themed distractions.

### Summary
- **Hooters** and **Buffalo Wild Wings** are the most relevant alternatives, offering similar sports bar experiences with potentially better food and service than Twin Peaks.
- **The Tilted Kilt** provides a comparable themed experience but may differ in menu offerings.
- **The Foxhole** stands out for its local charm and quality food, making it a great option for those seeking a more traditional sports bar vibe.

Overall, if the user is looking for a better dining experience than Twin Peaks, **Buffalo Wild Wings** and **Hooters** would be the top recommendations, followed by **The Tilted Kilt** and **The Foxhole** for those who prefer a more local or casual atmosphere.


#### Referenced Paragraphs
**Twin Peaks**

> Twin Peaks is your typical guys bar, way above Hooters, and on par with tilted kilt.  The thing that puts it over the top is the ICE cold beer served at 29 degrees. (Review ID: <REVIEW_ID>jAvNoTtB1x2LELl_-7y8kg</REVIEW_ID>)

> Twin Peaks is your typical guys bar, way above Hooters, and on par with tilted kilt.  The thing that puts it over the top is the ICE cold beer served at 29 degrees.   (Review ID: <REVIEW_ID>KsmnyFmghtOx_IbOG9RINg</REVIEW_ID>)

> Twin peaks has been open for a couple of months and I had heard good things about the food and the scenery and decided to check it out. I was told by some of my friends that it was substantially better than Hooters in both regards as well. 

We were seated faster than I expected and initially thought it was strange that we were sitting at a very long table along with others. The table was separated by a piece of wood with an ax in it.  (Review ID: <REVIEW_ID>l3jFrMXk3h99WlTv-q9gNw</REVIEW_ID>)

> ...If you happen to stumble into Twin Peaks expecting an otherworldly encounter with David Lynch, you're probably going to be disappointed. 

On the other hand, if you stumble into the brand new Indianapolis location of the Texas-based chain of "breastaurants" called Twin Peaks hoping to enjoy some good eats, good drinks, and scenic views, then you'll likely be incredibly happy. 

Founded in 2005 in Lewisville, Texas, Twin Peaks opened for the dinner hour starting on 12/8/14 at the former Loon Lake Lodge just East of Castleton at the corner of I-69 and East 82nd Street. 

With its servers all wearing the trademark Twin Peaks cleavage and midriff-baring red plaid tops and khaki shorts, the concept behind Twin Peaks may sound familiar but this variation on the familiar theme is far less kitschy and far more personable than other similarly themed restaurants.

The easiest way to describe Twin Peaks is sports bar meets hunting lodge meets bevy of beautiful women. For those who remember Loon L...

> If I were to describe twin peaks to someone, I would tell them it is an upscale Hooters.  The main difference between Hooters and twin peaks is the quality and the customer service. I'm not saying hooters is bad and does not have good service, but I would pick twin peaks over hooters any day of the week.    (Review ID: <REVIEW_ID>1Tn6LTyYfEwC0TSPsxohXw</REVIEW_ID>)

> ...Her service was over-the-top, too  (not once did I go lacking for anything), but two other things stood out in my mind as well.  While Twin Peaks is a sports-bar/restaurant, they're very family friendly.  That's unusual for many sports bars/restaurants I've visited. Singles, couples, families, and sports enthusiasts waiting to watch the games were all present. And I can't begin to describe how delicious the food was!  For lunch I had "Mom's Pot Roast," but to be honest mom never made it this great.  My dessert was a Twin Peaks Sundae that was amazing.  For my return trip at dinner, I specifically asked if Arielle was available.  I was informed I was in luck.  She had worked a longer shift than usual today.  I wanted to eat lighter, so I had a cup of Blonde Bean with Bacon Soup (delicious!) and asked Arielle specific questions about the items on the dessert menu I still hadn't tried.  (Again, there was no impatience or indifference.  She was warm and friendly and seemed eager to assist ...

> I'll NEVER go back to this dump. I'll never recommend this dump. Give me a good experience in a dump and and I'm good. A bad experience in a dump ? NO WAY. 

Get your reality readjusted, Twin Peaks. There are better places for food etc. unless you rate food according to breast exposure. I'll be back in Indy fir business but I'll never be back here. (Review ID: <REVIEW_ID>vne8OJzpDAQ6SmmRQG7nFQ</REVIEW_ID>)

> We stop by at Twin Peaks on our way to Indianapolis to watch the U of L and Purdue game. This was a cool place. It is a sports bar  that had waitresses that are dressed in Specialty little outfits kind of like a checkered shirt with shorts on it kind of fit the outdoor theme of it it was very like Cabela's meets Hooters is the best way I can put it at Giant TVs everywhere the bathrooms were phenomenal they had like TVs right at each year and I really enjoyed that bathroom so huge! And I mean huge just when you thought that a bathroom couldn't be that large it is.  (Review ID: <REVIEW_ID>4U3_8fv3RtU_lif2Ut5ESg</REVIEW_ID>)

> I'm a big sports bar guy and a sports bar food guy and don't expect a sports bar to be 5 star dining. Let me answer a question that people ask me when I said I ate there. "Is it okay to bring your wife and kids there to watch a game?" The servers do wear low cut shirts and shorts, but nothing that was any different than other themed sports bars. When I was there, there were groups of women together and also couples there.  (Review ID: <REVIEW_ID>grTir4NbamCVuxnzfa8sqw</REVIEW_ID>)

> You came here for the twin peaks.

Having said that, the food is on par with other sports bars, which means it's mediocre. So why four stars instead of the 2-3 stars I consistently give to sports bars with mediocre food (which is all of them)? Because twin peaks. (Review ID: <REVIEW_ID>_nUW-8x9p1n1VmOgW6ZPGA</REVIEW_ID>)

---

**`Source Node 1/10`**

**Node ID:** ad468cc4-df4f-4815-aceb-882c4d16e069<br>**Similarity:** 0.86279296875<br>**Text:** Twin Peaks is your typical guys bar, way above Hooters, and on par with tilted kilt.  The thing that puts it over the top is the ICE cold beer served at 29 degrees.<br>**Metadata:** {'review_id': 'jAvNoTtB1x2LELl_-7y8kg', 'user_id': 'jB-7q5FxZKho-wlRAzS2lA', 'business_id': 'seKihQKpGGnCeLuELRQPSQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2017-08-16T23:11:04', 'biz_name': 'Twin Peaks', 'biz_address': '6880 E 82nd St', 'biz_city': 'Indianapolis', 'biz_state': 'IN', 'biz_postal_code': '46250', 'biz_latitude': 39.9062952, 'biz_longitude': -86.0474634, 'biz_stars': 3.5, 'biz_review_count': 257, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "\'full_bar\'", \'Ambience\': "{u\'divey\': False, u\'hipster\': None, u\'casual\': True, u\'touristy\': None, u\'trendy\': None, u\'intimate\': False, u\'romantic\': False, u\'classy\': False, u\'upscale\': False}", \'BYOB\': None, \'BestNights\': "{u\'monday\': False, u\'tuesday\': True, u\'wednesday\': False, u\'thursday\': False, u\'friday\': True, u\'saturday\': True, u\'sunday\': False}", \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'False\', \'CoatCheck\': \'False\', \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': None, \'GoodForDancing\': \'False\', \'GoodForKids\': \'True\', \'GoodForMeal\': "{\'dessert\': None, \'latenight\': None, \'lunch\': True, \'dinner\': True, \'brunch\': False, \'breakfast\': False}", \'HappyHour\': \'True\', \'HasTV\': \'True\', \'Music\': "{\'dj\': False}", \'NoiseLevel\': "\'average\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'False\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'True\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': "u\'no\'", \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Sports Bars, American (New), American (Traditional), Nightlife, Bars, Restaurants'}<br>

...

**Node ID:** 73cab484-5136-49c5-a5ba-4e249e2af84f<br>**Similarity:** -0.33203125<br>**Text:** I'm a big sports bar guy and a sports bar food guy and don't expect a sports bar to be 5 star dining. Let me answer a question that people ask me when I said I ate there. "Is it okay to bring your wife and kids there to watch a game?" The servers do wear low cut shirts and shorts, but nothing that was any different than other themed sports bars. When I was there, there were groups of women together and also couples there.<br>**Metadata:** {'review_id': 'grTir4NbamCVuxnzfa8sqw', 'user_id': 'sZh1Pp958CUE5Tv3y5xtAQ', 'business_id': 'seKihQKpGGnCeLuELRQPSQ', 'review_stars': 4, 'useful': 3, 'funny': 1, 'cool': 2, 'date': '2015-11-04T16:52:57', 'biz_name': 'Twin Peaks', 'biz_address': '6880 E 82nd St', 'biz_city': 'Indianapolis', 'biz_state': 'IN', 'biz_postal_code': '46250', 'biz_latitude': 39.9062952, 'biz_longitude': -86.0474634, 'biz_stars': 3.5, 'biz_review_count': 257, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "\'full_bar\'", \'Ambience\': "{u\'divey\': False, u\'hipster\': None, u\'casual\': True, u\'touristy\': None, u\'trendy\': None, u\'intimate\': False, u\'romantic\': False, u\'classy\': False, u\'upscale\': False}", \'BYOB\': None, \'BestNights\': "{u\'monday\': False, u\'tuesday\': True, u\'wednesday\': False, u\'thursday\': False, u\'friday\': True, u\'saturday\': True, u\'sunday\': False}", \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'False\', \'CoatCheck\': \'False\', \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': None, \'GoodForDancing\': \'False\', \'GoodForKids\': \'True\', \'GoodForMeal\': "{\'dessert\': None, \'latenight\': None, \'lunch\': True, \'dinner\': True, \'brunch\': False, \'breakfast\': False}", \'HappyHour\': \'True\', \'HasTV\': \'True\', \'Music\': "{\'dj\': False}", \'NoiseLevel\': "\'average\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'False\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'True\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': "u\'no\'", \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Sports Bars, American (New), American (Traditional), Nightlife, Bars, Restaurants'}<br>

---

**`Source Node 10/10`**

**Node ID:** ebafc58c-c02f-4b13-bc73-0a2429ebdbf2<br>**Similarity:** -0.45263671875<br>**Text:** You came here for the twin peaks.

Having said that, the food is on par with other sports bars, which means it's mediocre. So why four stars instead of the 2-3 stars I consistently give to sports bars with mediocre food (which is all of them)? Because twin peaks.<br>**Metadata:** {'review_id': '_nUW-8x9p1n1VmOgW6ZPGA', 'user_id': 'KVN1eio-4QU3zlkPmZHRFQ', 'business_id': 'seKihQKpGGnCeLuELRQPSQ', 'review_stars': 4, 'useful': 0, 'funny': 3, 'cool': 2, 'date': '2017-01-13T02:56:31', 'biz_name': 'Twin Peaks', 'biz_address': '6880 E 82nd St', 'biz_city': 'Indianapolis', 'biz_state': 'IN', 'biz_postal_code': '46250', 'biz_latitude': 39.9062952, 'biz_longitude': -86.0474634, 'biz_stars': 3.5, 'biz_review_count': 257, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "\'full_bar\'", \'Ambience\': "{u\'divey\': False, u\'hipster\': None, u\'casual\': True, u\'touristy\': None, u\'trendy\': None, u\'intimate\': False, u\'romantic\': False, u\'classy\': False, u\'upscale\': False}", \'BYOB\': None, \'BestNights\': "{u\'monday\': False, u\'tuesday\': True, u\'wednesday\': False, u\'thursday\': False, u\'friday\': True, u\'saturday\': True, u\'sunday\': False}", \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'False\', \'CoatCheck\': \'False\', \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': None, \'GoodForDancing\': \'False\', \'GoodForKids\': \'True\', \'GoodForMeal\': "{\'dessert\': None, \'latenight\': None, \'lunch\': True, \'dinner\': True, \'brunch\': False, \'breakfast\': False}", \'HappyHour\': \'True\', \'HasTV\': \'True\', \'Music\': "{\'dj\': False}", \'NoiseLevel\': "\'average\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'False\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'True\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': "u\'no\'", \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Sports Bars, American (New), American (Traditional), Nightlife, Bars, Restaurants'}<br>

{'ad468cc4-df4f-4815-aceb-882c4d16e069': {'review_id': 'jAvNoTtB1x2LELl_-7y8kg',
  'user_id': 'jB-7q5FxZKho-wlRAzS2lA',
  'business_id': 'seKihQKpGGnCeLuELRQPSQ',
  'review_stars': 5,
  'useful': 0,
  'funny': 0,
  'cool': 0,
  'date': '2017-08-16T23:11:04',
  'biz_name': 'Twin Peaks',
  'biz_address': '6880 E 82nd St',
  'biz_city': 'Indianapolis',
  'biz_state': 'IN',
  'biz_postal_code': '46250',
  'biz_latitude': 39.9062952,
  'biz_longitude': -86.0474634,
  'biz_stars': 3.5,
  'biz_review_count': 257,
  'biz_is_open': 1,
  'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "\'full_bar\'", \'Ambience\': "{u\'divey\': False, u\'hipster\': None, u\'casual\': True, u\'touristy\': None, u\'trendy\': None, u\'intimate\': False, u\'romantic\': False, u\'classy\': False, u\'upscale\': False}", \'BYOB\': None, \'BestNights\': "{u\'monday\': False, u\'tuesday\': True, u\'wednesday\': False, u\'thursday\': False, u\'friday\': True, u\'saturday\': True, u\'sunday\': False}", \'BikeP

### Manually Curated
Ref: https://docs.llamaindex.ai/en/stable/examples/llama_dataset/ragdataset_submission_template/#1c-creating-a-labelledragdataset-from-scratch-with-manually-constructed-examples

In [62]:
response_curated_eval_dataset = response_evaluator.generate_curated_dataset(cfg)

2024-08-17 08:19:19.210 | INFO     | src.run.eval.response:generate_curated_dataset:125 - Persisting curated response eval dataset at data/027_auto_retrieval/response_curated_eval_dataset.json...


In [63]:
response_curated_eval_prediction_dataset = await response_curated_eval_dataset.amake_predictions_with(
    predictor=query_engine, batch_size=cfg.batch_size, show_progress=True
)

Batch processing of predictions:   0%|                                                                                                                                                                                                | 0/1 [00:00<?, ?it/s]

Generated queries:
recommended restaurants Santa Barbara nearby
"I recently visited Santa Barbara and discovered some amazing restaurants. The seafood at The Boathouse was fresh and delicious, while the ambiance at Finch & Fork was perfect for a cozy dinner. I highly recommend checking out these spots!"


Batch processing of predictions:   0%|                                                                                                                                                                                                | 0/1 [00:00<?, ?it/s]

Generated queries:
sports bars dining experience indianapolis better than Twin Peaks
"I visited a sports bar in Indianapolis that had an amazing atmosphere and delicious food. The menu was diverse, and the service was top-notch, making it a great alternative to Twin Peaks."


Batch processing of predictions: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:12<00:00, 12.70s/it]


In [64]:
response_curated_mean_scores_df, response_curated_deep_eval_df = response_evaluator.evaluate_labelled_rag_dataset(
    response_curated_eval_dataset,
    response_curated_eval_prediction_dataset,
    dataset_name="curated",
    judge_model=cfg.eval_cfg.response_eval_llm_model,
    cache_dp=cfg.notebook_cache_dp
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:11<00:00,  5.76s/it]


In [65]:
response_curated_mean_scores_df

rag,base_rag
metrics,
mean_correctness_score,4.00
mean_relevancy_score,1.00
mean_faithfulness_score,1.00
mean_context_relevancy_score,0.25


In [66]:
response_curated_deep_eval_df

,query,answer,relevancy_score,correctness_score,faithfulness_score,context_relevancy_score,contexts
0,What are the recommended restaurants nearby Sa...,"\nBased on the provided information, the top r...",1.0,4.0,1.0,0.5,[Finally some good food in downtown Santa Barb...
1,What are some sports bars in Indianapolis that...,"\nBased on the provided information, here are ...",1.0,4.0,1.0,0.0,"[Twin Peaks is your typical guys bar, way abov..."


#### Answers for target questions

In [67]:
from IPython.display import Markdown

In [68]:
from src.run.eval.manual_eval_dataset import MANUAL_EVAL_QA

In [69]:
for question, expected_answer in MANUAL_EVAL_QA:
    display(Markdown(f"\n\n### Question: {question}\n"))
    response = query_engine.query(question)
    display_response(response)



### Question: What are the recommended restaurants nearby Santa Barbara?


Generated queries:
recommended restaurants Santa Barbara
"I recently visited Santa Barbara and had an amazing meal at The Lark. The ambiance was fantastic, and the menu featured a great selection of local dishes. I also enjoyed dinner at Convivo, which offered stunning ocean views and delicious seafood."


**`Final Response:`** Based on the provided information, the top recommendation for restaurants in Santa Barbara is:

### 1. Helena Avenue Bakery
- **Address:** 131 Anacapa St, Ste C, Santa Barbara, CA
- **Categories:** Food, Restaurants, Salad, Coffee & Tea, Breakfast & Brunch, Sandwiches, Bakeries
- **Review Stars:** 5

**Strengths:**
- **Diverse Menu:** Helena Avenue Bakery offers a wide range of options including breakfast items, sandwiches, salads, and baked goods, making it suitable for various meal times and preferences.
- **High Ratings:** All reviews are rated 5 stars, indicating exceptional quality and customer satisfaction.
- **Location:** It is conveniently located within walking distance to the beach, making it a great stop for tourists exploring the area.
- **Dietary Accommodations:** The bakery is noted for being accommodating to dietary restrictions, which is a plus for families or groups with diverse needs.
- **Outdoor Seating:** The availability of outdoor seating, especially with safety measures in place, enhances the dining experience.

**Comparison:**
- **For Tourists:** Helena Avenue Bakery is highly recommended for tourists due to its excellent food quality, variety, and proximity to attractions like the beach. The positive reviews emphasize its appeal to visitors looking for a memorable dining experience.
- **For Locals:** While some reviews suggest it may be more appealing to tourists, the consistent praise for the food quality and service indicates that locals could also enjoy it, especially for special occasions or weekend brunches.

### Summary
Helena Avenue Bakery stands out as the top recommendation for restaurants in Santa Barbara due to its exceptional food, accommodating service, and prime location. It caters well to both tourists and locals, making it a versatile choice for anyone looking to enjoy a meal in the area. If you're in Santa Barbara, this bakery is a must-visit for a delightful dining experience.


#### Referenced Paragraphs
**Helena Avenue Bakery**

> Finally some good food in downtown Santa Barbara.  Everything on the menu is great and could not recommend more.  (Review ID: <REVIEW_ID>8qeD6EhCxUEBj1dO4J4KfA</REVIEW_ID>)

> If you're a tourist and spending the weekend in Santa Barbara, I'd recommend to give this place a go for sure. If you're a local, then not so much.  (Review ID: <REVIEW_ID>Mqh2anPpP_vvLtqCAk_1Xw</REVIEW_ID>)

> Another gem in Santa Barbara! Love the location, walking distance to the beach.  (Review ID: <REVIEW_ID>_S_Lnze72utd6e95FSnw6A</REVIEW_ID>)

> Can not recommend enough. Coffee, mocha, morning bun (warmed), breakfast biscuit sando, avocado toast, yes I had all of this and it was amazing. If I lived next door I would be 400lbs. I love this place!  (Review ID: <REVIEW_ID>74usfGu09q-ftJJd3bvmfg</REVIEW_ID>)

> This place was recommended by a foodie blogger.  (Review ID: <REVIEW_ID>iaQbi8ms6wHYYUu83QgCgQ</REVIEW_ID>)

> O....M....G!!!! I'm so glad this place is in Santa Barbara and not my hometown, because I would never stop eating. Their jams, scones, bagels, breads...delish!!!  (Review ID: <REVIEW_ID>8uAtrjUhdq5nFeHXllueBA</REVIEW_ID>)

> Would definitely recommend. (Review ID: <REVIEW_ID>GASDiG6rXbv390uwXvhQ1w</REVIEW_ID>)

> They offered outdoor seating which they disinfected after each attendee, and their hot chocolate was yummy. Definitely would recommend them, especially if you're in Santa Barbara and looking for a gluten-free breakfast at a cafe. (Review ID: <REVIEW_ID>FwB2dNaeap2oORCc88ubUQ</REVIEW_ID>)

> First time in Santa Barbara and we came upon this cute spot! Our family has a lot of dietary restrictions and our server was so knowledgable and helpful guiding us through the menu. Highly recommend!  (Review ID: <REVIEW_ID>UcTKZYl3_wydLoQCSx4Rwg</REVIEW_ID>)

> Highly recommend! (Review ID: <REVIEW_ID>7eTevb0NOzSg_CPrfi2o6w</REVIEW_ID>)



### Question: What are some sports bars in Indianapolis that offer a better dining experience than Twin Peaks?


Generated queries:
sports bars dining experience indianapolis better than Twin Peaks
"I visited a sports bar in Indianapolis that had an amazing atmosphere and delicious food. The service was top-notch, and they offered a great selection of beers. It was definitely a step up from Twin Peaks!"


**`Final Response:`** Based on the provided information, here are some recommendations for sports bars in Indianapolis that may offer a better dining experience than Twin Peaks, ranked by relevance:

### 1. **Hooters**
   - **Strengths**: Hooters is often compared to Twin Peaks, and while some reviews suggest that Twin Peaks offers a better experience, Hooters has a long-standing reputation for its wings and casual dining atmosphere. It may provide a more traditional sports bar experience with a focus on food quality and service.
   - **Comparison**: Hooters is a direct competitor to Twin Peaks, and while it may not have the same level of customer service or ambiance as Twin Peaks, it is a well-known option for sports fans looking for a casual dining experience.

### 2. **Tilted Kilt**
   - **Strengths**: Similar to Twin Peaks, Tilted Kilt offers a themed dining experience with a focus on sports and a lively atmosphere. The food quality and service are often highlighted positively, making it a solid alternative for those seeking a better dining experience.
   - **Comparison**: Tilted Kilt is on par with Twin Peaks in terms of theme and service but may offer a more diverse menu and a different ambiance that some patrons might prefer.

### 3. **Buffalo Wild Wings**
   - **Strengths**: Known for its extensive menu of wings and a variety of sauces, Buffalo Wild Wings is a popular choice for sports enthusiasts. The atmosphere is vibrant, with numerous TVs for game viewing, and it often has a family-friendly environment.
   - **Comparison**: While Buffalo Wild Wings may not have the same themed experience as Twin Peaks, it excels in food variety and quality, making it a better option for those prioritizing dining over the themed atmosphere.

### 4. **The Sports Page**
   - **Strengths**: This local sports bar is known for its laid-back atmosphere and good food. It offers a variety of American fare and is often praised for its friendly service and community vibe.
   - **Comparison**: The Sports Page may not have the same level of themed decor as Twin Peaks, but it provides a more relaxed dining experience with a focus on quality food and service.

### Summary Comparison:
- **Hooters** and **Tilted Kilt** are direct competitors to Twin Peaks, offering similar themed experiences but with varying levels of food quality and service. They are ideal for those looking for a comparable atmosphere but potentially better dining experiences.
- **Buffalo Wild Wings** stands out for its food variety and casual sports bar vibe, making it a great choice for those who prioritize food over the themed experience.
- **The Sports Page** offers a more traditional sports bar experience with a focus on community and quality food, making it suitable for those looking for a relaxed dining atmosphere.

In conclusion, if the user is looking for a better dining experience than Twin Peaks, **Buffalo Wild Wings** and **The Sports Page** may be the best options, while **Hooters** and **Tilted Kilt** provide similar themed experiences with potentially better service and food quality.


#### Referenced Paragraphs
**Twin Peaks**

> Twin Peaks is your typical guys bar, way above Hooters, and on par with tilted kilt.  The thing that puts it over the top is the ICE cold beer served at 29 degrees. (Review ID: <REVIEW_ID>jAvNoTtB1x2LELl_-7y8kg</REVIEW_ID>)

> Twin Peaks is your typical guys bar, way above Hooters, and on par with tilted kilt.  The thing that puts it over the top is the ICE cold beer served at 29 degrees.   (Review ID: <REVIEW_ID>KsmnyFmghtOx_IbOG9RINg</REVIEW_ID>)

> Twin peaks has been open for a couple of months and I had heard good things about the food and the scenery and decided to check it out. I was told by some of my friends that it was substantially better than Hooters in both regards as well. 

We were seated faster than I expected and initially thought it was strange that we were sitting at a very long table along with others. The table was separated by a piece of wood with an ax in it.  (Review ID: <REVIEW_ID>l3jFrMXk3h99WlTv-q9gNw</REVIEW_ID>)

> ...If you happen to stumble into Twin Peaks expecting an otherworldly encounter with David Lynch, you're probably going to be disappointed. 

On the other hand, if you stumble into the brand new Indianapolis location of the Texas-based chain of "breastaurants" called Twin Peaks hoping to enjoy some good eats, good drinks, and scenic views, then you'll likely be incredibly happy. 

Founded in 2005 in Lewisville, Texas, Twin Peaks opened for the dinner hour starting on 12/8/14 at the former Loon Lake Lodge just East of Castleton at the corner of I-69 and East 82nd Street. 

With its servers all wearing the trademark Twin Peaks cleavage and midriff-baring red plaid tops and khaki shorts, the concept behind Twin Peaks may sound familiar but this variation on the familiar theme is far less kitschy and far more personable than other similarly themed restaurants.

The easiest way to describe Twin Peaks is sports bar meets hunting lodge meets bevy of beautiful women. For those who remember Loon L...

> If I were to describe twin peaks to someone, I would tell them it is an upscale Hooters.  The main difference between Hooters and twin peaks is the quality and the customer service. I'm not saying hooters is bad and does not have good service, but I would pick twin peaks over hooters any day of the week.    (Review ID: <REVIEW_ID>1Tn6LTyYfEwC0TSPsxohXw</REVIEW_ID>)

> ...Her service was over-the-top, too  (not once did I go lacking for anything), but two other things stood out in my mind as well.  While Twin Peaks is a sports-bar/restaurant, they're very family friendly.  That's unusual for many sports bars/restaurants I've visited. Singles, couples, families, and sports enthusiasts waiting to watch the games were all present. And I can't begin to describe how delicious the food was!  For lunch I had "Mom's Pot Roast," but to be honest mom never made it this great.  My dessert was a Twin Peaks Sundae that was amazing.  For my return trip at dinner, I specifically asked if Arielle was available.  I was informed I was in luck.  She had worked a longer shift than usual today.  I wanted to eat lighter, so I had a cup of Blonde Bean with Bacon Soup (delicious!) and asked Arielle specific questions about the items on the dessert menu I still hadn't tried.  (Again, there was no impatience or indifference.  She was warm and friendly and seemed eager to assist ...

> I'll NEVER go back to this dump. I'll never recommend this dump. Give me a good experience in a dump and and I'm good. A bad experience in a dump ? NO WAY. 

Get your reality readjusted, Twin Peaks. There are better places for food etc. unless you rate food according to breast exposure. I'll be back in Indy fir business but I'll never be back here. (Review ID: <REVIEW_ID>vne8OJzpDAQ6SmmRQG7nFQ</REVIEW_ID>)

> We stop by at Twin Peaks on our way to Indianapolis to watch the U of L and Purdue game. This was a cool place. It is a sports bar  that had waitresses that are dressed in Specialty little outfits kind of like a checkered shirt with shorts on it kind of fit the outdoor theme of it it was very like Cabela's meets Hooters is the best way I can put it at Giant TVs everywhere the bathrooms were phenomenal they had like TVs right at each year and I really enjoyed that bathroom so huge! And I mean huge just when you thought that a bathroom couldn't be that large it is.  (Review ID: <REVIEW_ID>4U3_8fv3RtU_lif2Ut5ESg</REVIEW_ID>)

> I'm a big sports bar guy and a sports bar food guy and don't expect a sports bar to be 5 star dining. Let me answer a question that people ask me when I said I ate there. "Is it okay to bring your wife and kids there to watch a game?" The servers do wear low cut shirts and shorts, but nothing that was any different than other themed sports bars. When I was there, there were groups of women together and also couples there.  (Review ID: <REVIEW_ID>grTir4NbamCVuxnzfa8sqw</REVIEW_ID>)

> You came here for the twin peaks.

Having said that, the food is on par with other sports bars, which means it's mediocre. So why four stars instead of the 2-3 stars I consistently give to sports bars with mediocre food (which is all of them)? Because twin peaks. (Review ID: <REVIEW_ID>_nUW-8x9p1n1VmOgW6ZPGA</REVIEW_ID>)

### Error Analysis

In [70]:
response_curated_eval_dataset_dict = dict()
for example in response_curated_eval_dataset.examples:
    response_curated_eval_dataset_dict[example.query] = {
        "reference_answer": example.reference_answer,
        "reference_contexts": example.reference_contexts,
    }

In [71]:
response_curated_deep_eval_df.sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])

,query,answer,relevancy_score,correctness_score,faithfulness_score,context_relevancy_score,contexts
0,What are the recommended restaurants nearby Sa...,"\nBased on the provided information, the top r...",1.0,4.0,1.0,0.5,[Finally some good food in downtown Santa Barb...
1,What are some sports bars in Indianapolis that...,"\nBased on the provided information, here are ...",1.0,4.0,1.0,0.0,"[Twin Peaks is your typical guys bar, way abov..."


In [72]:
response_curated_eval_irrelevance_df = (
    response_curated_deep_eval_df
    .loc[lambda df: df['relevancy_score'].lt(1)]
    .sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])
)

for i, row in response_curated_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n==============Error #{i+1}===============\n\n")
    print(f"Query:\n{row.query}\n")
    contexts = '\n\n'.join(row.contexts)
    print(f"Context:\n{contexts}\n")
    print(f"Answer:\n{row.answer}\n----\n")
    expected = response_curated_eval_dataset_dict.get(row.query)
    if not expected:
        logger.error(f"Could not find query {row.query} in synthetic_response_eval_dataset_dict!")
        continue
    expected_answer = expected['reference_answer']
    print(f"Expected Answer:\n{expected_answer}\n")
    expected_contexts = expected['reference_contexts']
    print(f"Expected Contexts:\n{expected_contexts}\n")

# Persist run metadata

In [73]:
from src.run.utils import parse_collect_log, flatten_dict

In [74]:
collect_info = parse_collect_log(collect_fp)

In [75]:
if ARGS.LOG_TO_MLFLOW:
    import mlflow

    logger.info("Logging [COLLECT] info to MLflow...")
    mlflow.log_params(collect_info)
    logger.info("Logging config to MLflow...")
    mlflow.log_params(flatten_dict(cfg.model_dump(), "cfg", sep='.'))
    logger.info(f"Logging Retrieval Synthetic Eval Results to MLflow...")
    retrieval_evaluator.log_to_mlflow(cfg)
    logger.info(f"Logging Response Eval Results to MLflow...")
    try:
        response_evaluator.log_to_mlflow(
            cfg,
            'synthetic',
            response_synthetic_mean_scores_df,
            response_synthetic_deep_eval_df
        )
        response_evaluator.log_to_mlflow(
            cfg,
            'curated',
            response_curated_mean_scores_df,
            response_curated_deep_eval_df
        )
    except:
        ...

2024-08-17 08:20:14.122 | INFO     | __main__:<module>:4 - Logging [COLLECT] info to MLflow...
2024-08-17 08:20:14.151 | INFO     | __main__:<module>:6 - Logging config to MLflow...
/home/dvquys/frostmourne/study/vietai-genai03/final-project/review-rec-bot/.venv/lib/python3.11/site-packages/pydantic/main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
2024-08-17 08:20:14.185 | INFO     | __main__:<module>:8 - Logging Retrieval Synthetic Eval Results to MLflow...
2024-08-17 08:20:14.528 | INFO     | __main

# Clean up

In [76]:
if ARGS.LOG_TO_MLFLOW:
    mlflow.end_run()

# Chat Bot Quick Test

In [85]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

from src.svc.availability.availability_check import ReservationService

In [86]:
query_engine_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="reco_review",
        description=(
            "useful for when you want to find places to visit"
            " based on end-user reviews"
        ),
    ),
)

opening_hours_db = data.set_index('business_id')['biz_hours'].dropna().to_dict()
biz_name_id_mapper = data.set_index('biz_name')['business_id'].dropna().to_dict()
rez_tool = ReservationService(opening_hours_db, biz_name_id_mapper)

tools = [query_engine_tool] + rez_tool.to_tool_list()

In [87]:
from llama_index.agent.openai import OpenAIAgent

agent = OpenAIAgent.from_tools(tools, verbose=True)

In [88]:
response = agent.chat("hi, i am quy")
print(str(response))

response = agent.chat(
    "Where to find some good cold brew coffee?"
)
print(str(response))

response = agent.chat(
    "where is it?"
)
print(str(response))

response = agent.chat(
    "what is the best time to visit?"
)
print(str(response))

Added user message to memory: hi, i am quy
Hello Quy! How can I assist you today?
Added user message to memory: Where to find some good cold brew coffee?
=== Calling Function ===
Calling function: reco_review with args: {"input":"cold brew coffee"}
Generated queries:
cold brew coffee
"I recently tried a new café that specializes in cold brew coffee, and I was amazed by the smooth flavor and rich aroma. They also offer various blends and seasonal flavors that are perfect for any coffee lover."
Got output: 
Based on your interest in cold brew coffee, here are several recommendations ranked by relevance:

### 1. **Helena Avenue Bakery**
   - **Location:** 131 Anacapa St, Ste C, Santa Barbara, CA
   - **Rating:** 5 stars
   - **Strengths:** Helena Avenue Bakery is highly praised for its cold brew coffee, which has received multiple mentions in reviews for being exceptionally good. The bakery also offers a variety of delicious breakfast options, including sandwiches and pastries, making it 

In [89]:
response = agent.chat(
    "Can I book for 5 pax on August 15 at 8 AM?"
)
print(str(response))

Added user message to memory: Can I book for 5 pax on August 15 at 8 AM?
=== Calling Function ===
Calling function: get_business_id with args: {"biz_name": "Helena Avenue Bakery"}
Got output: IDtLPgUrqorrpqSLdfMhZQ

=== Calling Function ===
Calling function: get_business_id with args: {"biz_name": "Caviar & Bananas"}
Got output: lk9IwjZXqUMqqOhM774DtQ

=== Calling Function ===
Calling function: get_business_id with args: {"biz_name": "Wawa"}
Got output: ppFCk9aQkM338Rgwpl2F5A

=== Calling Function ===
Calling function: get_business_id with args: {"biz_name": "Barnes & Noble Booksellers"}
Got output: qhDdDeI3K4jy2KyzwFN53w

=== Calling Function ===
Calling function: check_availability with args: {"business_id": "IDtLPgUrqorrpqSLdfMhZQ", "date_time": "2023-08-15T08:00:00", "pax": 5}
Got output: True

=== Calling Function ===
Calling function: check_availability with args: {"business_id": "lk9IwjZXqUMqqOhM774DtQ", "date_time": "2023-08-15T08:00:00", "pax": 5}
Got output: False

=== Callin

In [77]:
agent.reset()

# Chatbot UI

# Archive